In [49]:
import numpy as np
import pandas as pd
import requests


from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
tqdm.pandas()

import matplotlib
from matplotlib.pylab import *
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits.axes_grid1 import host_subplot
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
%matplotlib inline 


# importing movie py libraries
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage

import seaborn as sns

from datetime import datetime


pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.7f}'.format

In [50]:
# from terra_sdk.client.lcd import LCDClient
# terra = LCDClient(chain_id="columbus-5", url="https://lcd.terra.dev")

In [51]:
# get Market Module Parameters
# terra.market.parameters()
# terra.market.terra_pool_delta()
# terra.market.swap_rate()

# get current Excahnge rates as provided by the Oracle
# terra.oracle.exchange_rates()

# terra.ibc.parameters()
# terra.ibc_transfer.parameters()



In [52]:
# import requests
# pr = requests.get('https://fcd.terra.dev/v1/market/price?denom=uusd&interval=1h').json()

In [53]:
# latest_height = terra.tendermint.block_info()["block"]["header"]["height"]

# terra.gov.proposal(100, )

In [54]:
# imageio.plugins.ffmpeg.download()
# # pip install imageio==2.4.1

In [55]:
# pip install imageio==2.4.1
# pip3 install --upgrade imageio-ffmpeg

# Introduction


## Terra’s Mission 

Terra was created with the goal to power the innovation of money by building a price-stable cryptocurrency that can be used as a means of payment and store of value at a global scale. 

The Terra protocol acted as an algorithmic entity (the de-central bank) that is responsible for maintaining this peg. 


## How Terra worked under the hood ? 

To achieve this, it implemented the underlying Luna <> Terra coins mint / burn algorithm within the core protocol via the [market module](https://github.com/terra-money/classic-core/tree/main/x/market/spec). The underlying swaps were processed in International Monetary Fund’s  Special Drawing Rights (SDR), which is a basket of multiple currencies with the official SDR rate published by the IMF on a daily basis, allowing arbitragers, market makers, and Terra to closely align the exchange rate to the peg.

The mint / burn swap mechanism is implemented as a constant Product market-making algorithm and was aimed to be designed as a function of the parity between the off-chain liquidity of LUNA/UST and the on-chain liquidity parameters for the redemption of UST/LUNA. However, it failed to behave as expected triggering a “death spiral” bank run characteristic of traditional endogenous collateral models. You can refer to research article for Jump crypto covering the terra crash more in detail [here](https://jumpcrypto.com/the-depegging-of-ust/).




## Understanding Terra's Market Module

The **base market** starts out with two liquidity pools of equal sizes, 

- one representing Terra Stablecoins (all denominations), represented by `terra_pool` parameter
- ananother representing Luna, represented by `luna_pool` parameter


The **base market** is initialized by the parameter `BasePool` (denominated in [SDR](https://en.wikipedia.org/wiki/Special_drawing_rights) ), which defines the initial size of the Terra and Luna liquidity pools.

- The `base_pool` parameter is currently set to `100000000000000`, representing 100 Million SDR. 

Any market swap between Luna <> Terra stablecoins leads to a state transition of their `total_supply` parameter value based via the `mint` / `burn` functions, and this net transition is effectively also captured in the number variable `delta` which tracks net change in the sizes of the two pools, representing the deviation of the Terra pool from its base size in units µSDR.


### Swap Procedure

1. Market module receives `MsgSwap` message and performs basic validation checks

2. Calculate ask and spread using `k.ComputeSwap()`

3. Update TerraPoolDelta with `k.ApplySwapToPool()`

4. Transfer OfferCoin from account to module using `supply.SendCoinsFromAccountToModule()`

5. Burn offered coins, with `supply.BurnCoins()`.

6. Let `fee = spread * ask`, this is the `spread fee`.

7. Mint `ask - fee coins` of AskDenom with `supply.MintCoins()`. This implicitly applies the spread fee as the fee coins are burned.

8. Send newly minted coins to trader with `supply.SendCoinsFromModuleToAccount()`

9. Emit swap event to publicize swap and record spread fee


At the end of each block, the market module will attempt to "replenish" the pools by decreasing the magnitude of between the Terra and Luna pools. The rate at which the pools will be replenished toward equilibrium is set by the parameter `PoolRecoveryPeriod`, with lower periods meaning lower sensitivity to trades, meaning previous trades are more quickly forgotten and the market is able to offer more liquidity.

This mechanism ensures liquidity and acts as a sort of low-pass filter, allowing for the `spread fee` (which is a function of `TerraPoolDelta`) to drop back down when there is a change in demand, hence necessary change in supply which needs to be absorbed.

#### Seigniorage
For Luna swaps into Terra, the Luna that recaptured by the protocol is burned and is called seigniorage -- the value generated from issuing new Terra. At the end of the epoch, the total seigniorage for the epoch will be calculated and reintroduced into the economy as ballot rewards for the exchange rate oracle and to the community pool by the Treasury module, described more fully [here](https://github.com/terra-money/classic-core/blob/main/x/treasury/spec/README.md).


Jump crypto has been tracking the performance of the market module params and recommending updates via Governance as visible via the proposals here. Brief pointers from each proposal are mentioned here - 


1. [**Jan 21 → Proposal to update market module parameters by Jump crypto**](https://classic-agora.terra.money/t/terra-on-chain-liquidity-parameters/305). 

    This proposal was followed by another one by Do Kown [here](https://classic-agora.terra.money/t/tip-36-further-improvements-to-liquidity-parameters/372) proposing further increasing market module's parameters to `Basepool 7M → 13M SDT` `PoolRecoveryPeriod 200 → 130 blocks` which will roughly terra's stablecoin minting capacity. 



2. [**May 21 →  Proposal by Jump crypto to update market module parameters**](https://classic-agora.terra.money/t/liquidity-parameters-2/1175)

     Jump briefly discussed the oraclre attack possible to be executed on terra in-case on chain liquidity driven off the oracle price is larger than liquidity off chain. Short mention on how one possible solution to mitigate this can be splitting out the `TerraPoolDelta` parameter into `TerraPoolDeltaBid` and `TerraPoolDeltaAsk`. 
    Proposed parameter updates were the following - 

    - Increase BasePool size to 32,500,000 SDR
    - Reduce PoolRecoveryPeriod to 49 blocks
    - Split out TerraPoolDelta into TerraPoolDeltaBid and TerraPoolDeltaAsk.



3. [**Jan 29, 2022 →  Proposal by Jump crypto to update market module parameters**](https://classic-agora.terra.money/t/liquidity-parameters-3/3895)

    Proposed parameter updates were the following - 

    - Increase BasePool size to 50,000,000 SDR
    - Reduce PoolRecoveryPeriod to 36 blocks


5. [**Jan 29, 2022 →  Proposal by Jump crypto for BTC reserve pools**](https://classic-agora.terra.money/t/bitcoin-reserve-pool/5259/23) --> Most of the BTC accumulated for deploying the BTC reserve pool was sold during the May, 22 crash and is not relevent for our analysis. 


## Simulated Analysis of Terra Classic's May 2022 Crash

This notebook contains busines logic for analyzing how terra classic's market module operated during the May 2022 crash.

- To do this, we have implemented terra's market module, oracle module and bank module as separate python classes. 

For simulation purposes, we have taken average of provided oracle pricefeeds per block by validators (not taking into account their vote share) and used the average price as exchange rates for computing simulated swaps until the next oracle updates are provided. (Generally after 4-5 blocks)

- **UST's price in USD** : We have used the provided oracle exchange rates and LUNA <> UST swaps via Astroport data to calculate UST's actual price in USD terms via the following formula, 

                UST's price in USD = uusd exchange rate provided via oracle / UST per Luna traded on astroport


**Note - We only look into LUNA <> UST market swaps for our analysis.**




### Import dataset

1. Import all cleaned data files 

- **aggregated_exchange_rate_vote_txs_DF** : Contains averaged oracle prices as provided by validators during per block 
- **market_swap_txs_DF** : Contains mainnet txs involving Luna <> UST swaps via Terra's Market module
- **astroport_ust_luna_txs_DF** : Contains mainnet txs involving Luna <> UST swaps via Astroport
- **aggregated_market_swap_txs_DF** : Contains total UST / LUNA which got minted / burnt per block


In [57]:
# Get DataFrames from pre-processed .csv files

## Oracle prices dataset
aggregated_exchange_rate_vote_txs_DF = pd.read_csv("./terra_classic_dataset/latest/aggregated_exchange_rate_vote_txs_DF.csv") 

## LUNA <> UST Swaps via Astroport dataset
astroport_ust_luna_txs_DF = pd.read_csv("./terra_classic_dataset/latest/astroport_ust_luna_txs_DF.csv") 

## LUNA <> UST Market Module swaps which involve mint / burn
market_swap_txs_DF = pd.read_csv("./terra_classic_dataset/latest/market_swap_txs_DF.csv") 

## Market Swaps via Module :: Aggregated Trading Volumes / Supply contraction / expansion dynamics
aggregated_market_swap_txs_DF = pd.read_csv("./terra_classic_dataset/latest/aggregated_market_swap_txs_DF.csv") 


# Remove index column
aggregated_exchange_rate_vote_txs_DF.drop(['Unnamed: 0'],axis=1,inplace=True)
astroport_ust_luna_txs_DF.drop(['Unnamed: 0'],axis=1,inplace=True)
aggregated_market_swap_txs_DF.drop(['Unnamed: 0'],axis=1,inplace=True)
market_swap_txs_DF.drop(['Unnamed: 0'],axis=1,inplace=True)




**Data Cleaning :: Step 1 - Filter data for only common blocks**

In [58]:
# Get Block heights for which data is available and block height range for which simulation is to be executed

astroport_txs_DF_start_block = astroport_ust_luna_txs_DF.iloc[0]["BlockHeight"]
astroport_txs_DF_end_block = astroport_ust_luna_txs_DF.iloc[len(astroport_ust_luna_txs_DF.index) - 1]["BlockHeight"]
print(f"astroport_txs_DF || Start block = {astroport_txs_DF_start_block}  End block = {astroport_txs_DF_end_block} \
Total Blocks = {astroport_txs_DF_end_block - astroport_txs_DF_start_block}")

aggregated_market_swap_txs_DF_start_block = aggregated_market_swap_txs_DF.iloc[0]["BlockHeight"]
aggregated_market_swap_txs_DF_end_block = aggregated_market_swap_txs_DF.iloc[len(aggregated_market_swap_txs_DF.index) - 1]["BlockHeight"]
print(f"aggregated_market_swap_txs_DF || Start block = {aggregated_market_swap_txs_DF_start_block}  End block = {aggregated_market_swap_txs_DF_end_block} \
Total Blocks = {aggregated_market_swap_txs_DF_end_block - aggregated_market_swap_txs_DF_start_block}")

market_swap_txs_DF_start_block = market_swap_txs_DF.iloc[0]["BlockHeight"]
market_swap_txs_DF_end_block = market_swap_txs_DF.iloc[len(market_swap_txs_DF.index) - 1]["BlockHeight"]
print(f"market_swap_txs_DF || Start block = {market_swap_txs_DF_start_block}  End block = {market_swap_txs_DF_end_block} \
Total Blocks = {market_swap_txs_DF_end_block - market_swap_txs_DF_start_block}")

oracle_txs_DF_start_block = aggregated_exchange_rate_vote_txs_DF.iloc[0]["BlockHeight"]
oracle_txs_DF_end_block = aggregated_exchange_rate_vote_txs_DF.iloc[len(aggregated_exchange_rate_vote_txs_DF.index) - 1]["BlockHeight"]
print(f"oracle_txs_DF || Start block = {oracle_txs_DF_start_block}  End block = {oracle_txs_DF_end_block} \
Total Blocks = {oracle_txs_DF_end_block - oracle_txs_DF_start_block}")

START_BLOCK = int(max(astroport_txs_DF_start_block, max(market_swap_txs_DF_start_block, oracle_txs_DF_start_block)))
START_BLOCK = int(max(START_BLOCK, aggregated_market_swap_txs_DF_start_block))
END_BLOCK = int(min(astroport_txs_DF_end_block, min(market_swap_txs_DF_end_block, oracle_txs_DF_end_block)))
END_BLOCK = int(min(END_BLOCK, aggregated_market_swap_txs_DF_end_block))
print(f"\nCommon Range, Start block = {START_BLOCK} End block = {END_BLOCK} | Total blocks = {END_BLOCK - START_BLOCK}")



astroport_txs_DF || Start block = 7117990  End block = 7638049 Total Blocks = 520059
aggregated_market_swap_txs_DF || Start block = 6960461.0  End block = 7639888.0 Total Blocks = 679427.0
market_swap_txs_DF || Start block = 6960461  End block = 7639889 Total Blocks = 679428
oracle_txs_DF || Start block = 6958375.0  End block = 7639899.0 Total Blocks = 681524.0

Common Range, Start block = 7117990 End block = 7638049 | Total blocks = 520059


In [60]:
import datetime
import calendar

# START BLOCK : 2022.04.04 19:25:06+04:00   -   https://finder.terra.money/classic/blocks/7117990
start_block_time = datetime.datetime(2022, 4, 4, 19, 25, 6)
ttuple = start_block_time.timetuple()
start_block_timestamp = calendar.timegm(ttuple)

# END BLOCK : 2022.05.13 16:03:06+04:00   -   https://finder.terra.money/classic/blocks/7638049
end_block_time = datetime.datetime(2022, 5, 15, 20, 52, 8)
ttuple = end_block_time.timetuple()
end_block_timestamp = calendar.timegm(ttuple)

difference_timestamps = end_block_timestamp - start_block_timestamp
average_block_time = difference_timestamps / (END_BLOCK - START_BLOCK)

print(f"Start block time : {start_block_time} | Timestamp = {start_block_timestamp}")
print(f"End block time : {end_block_time} | Timestamp = {end_block_timestamp}")
print(f"Average block time = {average_block_time}")



Start block time : 2022-04-04 19:25:06 | Timestamp = 1649100306
End block time : 2022-05-15 20:52:08 | Timestamp = 1652647928
Average block time = 6.821576013490777


In [61]:
# Shorten dataset for faster execution by taking only dataset for only common blocks
aggregated_exchange_rate_vote_txs_DF = aggregated_exchange_rate_vote_txs_DF.loc[(aggregated_exchange_rate_vote_txs_DF['BlockHeight'] >= START_BLOCK) & (aggregated_exchange_rate_vote_txs_DF['BlockHeight'] <= END_BLOCK)]
astroport_ust_luna_txs_DF = astroport_ust_luna_txs_DF.loc[(astroport_ust_luna_txs_DF['BlockHeight'] >= START_BLOCK) & (astroport_ust_luna_txs_DF['BlockHeight'] <= END_BLOCK)]
aggregated_market_swap_txs_DF = aggregated_market_swap_txs_DF.loc[(aggregated_market_swap_txs_DF['BlockHeight'] >= START_BLOCK) & (aggregated_market_swap_txs_DF['BlockHeight'] <= END_BLOCK)]
market_swap_txs_DF = market_swap_txs_DF.loc[(market_swap_txs_DF['BlockHeight'] >= START_BLOCK) & (market_swap_txs_DF['BlockHeight'] <= END_BLOCK)]



**crash_analysis_DF : DataFrame containing cleaned summarized data for each block**

- Replace 0 values with last valid provided oracle prices
- Create new DF by merging available DFs on BlockHeight
- Fill in na values for astroport prices for missing blocks with last valid values
- Drop duplicate rows based on BlockHeight value
- Calculate UST and LUNA exchange rate in USD terms

In [62]:
# Fill in missing Oracle prices values based on previous block inputs
aggregated_exchange_rate_vote_txs_DF.replace(to_replace = 0,  method='bfill', inplace=True)

# Merge DFs and select columns we need 
crash_analysis_DF = pd.merge(aggregated_exchange_rate_vote_txs_DF,aggregated_market_swap_txs_DF, on='BlockHeight',how='left')
crash_analysis_DF = pd.merge(crash_analysis_DF, astroport_ust_luna_txs_DF, on='BlockHeight',how='left')
crash_analysis_DF = crash_analysis_DF[["BlockHeight","uusd::minted","uusd::burnt","uluna::minted","uluna::burnt","uusd::mean","Price (luna/ust)","Price (ust/luna)"]]

# Fill in missing values via forward propoagating correct values
crash_analysis_DF["Price (luna/ust)"].fillna(method="ffill",inplace=True)
crash_analysis_DF["Price (ust/luna)"].fillna(method="ffill",inplace=True)

# Remove duplicates based on block height
crash_analysis_DF.drop_duplicates(subset=["BlockHeight"],inplace=True )

# Calculate UST's price in USD based on Astroport and Oracle data available
crash_analysis_DF["ExchangeRate::USD/UST"] = crash_analysis_DF.progress_apply(lambda x: x["uusd::mean"]*x["Price (luna/ust)"] , axis=1)

# Rename columns
crash_analysis_DF.rename(columns={"uusd::mean":"ExchangeRate::USD/LUNA","Price (luna/ust)":"ExchangeRate::LUNA/UST","Price (ust/luna)":"ExchangeRate::UST/LUNA"}, inplace=True)


100%|██████████| 520060/520060 [00:03<00:00, 172688.36it/s]


In [65]:
# crash_analysis_DF.tail(50)

**Set initial LUNA supply and UST supply and calculate the total supplies for each block**

In [66]:
# Create new columns for LUNA / UST Supply values
crash_analysis_DF['LUNA Supply'] = 0
crash_analysis_DF['UST Supply'] = 0


In [67]:
# Enter initial LUNA / UST Supply values, and timestamp
crash_analysis_DF.loc[0, 'LUNA Supply'] = 800_000_000     # 800 Million 
crash_analysis_DF.loc[0, 'UST Supply'] = 15_000_000_000   # 15 Billion

# We calculate estimated timestamp for a certain block, assuming block time ~ 7 sec
crash_analysis_DF.loc[0, 'timestamp'] = start_block_timestamp 

# Reset Index
crash_analysis_DF.reset_index(inplace=True)
crash_analysis_DF.drop('index', axis=1, inplace=True)

# Calculate new LUNA / UST total Supplies for each block
for i in tqdm_notebook(range(1, len(crash_analysis_DF))):
    crash_analysis_DF.loc[i, 'LUNA Supply'] = crash_analysis_DF.loc[i-1, 'LUNA Supply'] + crash_analysis_DF.loc[i, 'uluna::minted'] - crash_analysis_DF.loc[i, 'uluna::burnt']
    crash_analysis_DF.loc[i, 'UST Supply'] = crash_analysis_DF.loc[i-1, 'UST Supply'] + crash_analysis_DF.loc[i, 'uusd::minted'] - crash_analysis_DF.loc[i, 'uusd::burnt']
    crash_analysis_DF.loc[i, 'timestamp'] = crash_analysis_DF.loc[i-1, 'timestamp'] + average_block_time


# Calculate dateTime
crash_analysis_DF["dateTime"] = crash_analysis_DF.progress_apply(lambda x: datetime.datetime.fromtimestamp(int(x["timestamp"])) , axis=1)


  0%|          | 0/520059 [00:00<?, ?it/s]

100%|██████████| 520060/520060 [00:02<00:00, 199125.95it/s]


In [68]:
# Select only needed columns for visualization purposes from here on
crash_analysis_DF["timestamp"] = crash_analysis_DF.progress_apply(lambda x: int(x["timestamp"]) , axis=1)
crash_analysis_DF = crash_analysis_DF[["BlockHeight","timestamp","dateTime","ExchangeRate::USD/LUNA","ExchangeRate::UST/LUNA","ExchangeRate::USD/UST","ExchangeRate::LUNA/UST"\
                                       ,"uluna::minted","uluna::burnt","uusd::minted","uusd::burnt","LUNA Supply","UST Supply"]]



100%|██████████| 520060/520060 [00:02<00:00, 189598.58it/s]


In [69]:
crash_analysis_DF.head(5)

,BlockHeight,timestamp,dateTime,ExchangeRate::USD/LUNA,ExchangeRate::UST/LUNA,ExchangeRate::USD/UST,ExchangeRate::LUNA/UST,uluna::minted,uluna::burnt,uusd::minted,uusd::burnt,LUNA Supply,UST Supply
0,7117990,1649100306,2022-04-04 23:25:06,110.7717770,111.2217760,0.9959540,0.0089910,0.0000000,94.2895450,"10,434.7495680",0.0000000,"800,000,000.0000000","15,000,000,000.0000000"
1,7117991,1649100312,2022-04-04 23:25:12,110.7781370,111.2253395,0.9959793,0.0089908,0.0000000,0.0000000,0.0000000,0.0000000,"800,000,000.0000000","15,000,000,000.0000000"
2,7117992,1649100319,2022-04-04 23:25:19,110.7115950,111.2253395,0.9953810,0.0089908,0.0000000,0.0000000,0.0000000,0.0000000,"800,000,000.0000000","15,000,000,000.0000000"
3,7117993,1649100326,2022-04-04 23:25:26,110.7115950,111.2250797,0.9953834,0.0089908,0.0000000,103.4122460,"11,444.3323410",0.0000000,"799,999,896.5877540","15,000,011,444.3323402"
4,7117994,1649100333,2022-04-04 23:25:33,110.7115950,110.5567009,1.0014010,0.0090451,316.2638500,118.3069290,"13,092.6835660","35,000.0000000","800,000,094.5446750","14,999,989,537.0159054"


In [71]:
crash_analysis_DF.to_csv("./simulated_datasets/terra_crash_dataset.csv")

In [27]:
type(crash_analysis_DF.loc[0,"BlockHeight"])

numpy.int64

## TERRA CLASSIC : Simulating on-chain crash to gather delta, base_pool on-chain metrics.  


Here, we sequentially process each UST burn market swap tx via our implemented model to gather data for analytical and research purposes. 


Assumptions made  - 

- We have not taken into a/c delegation weights to compute oracle prices. An average has been taken of all the Oracle block txs for each block and the same prices have been used for the following blocks until the next update.
- delta is assumed to be 0 initially.
- Initial UST supply = 18 Billion
- Initial LUNA supply = 800 million
- Only UST burn txs have been considered



### Market Module base_pool parameter history
    -  2020-09-02 : 250,000 SDR to 625,000 SDR
    - 2021-02-10 : Basepool 7M → 13M SDT
    - 2022-05-23 : Increase BasePool size to 32,500,000 SDR
    - 2022-02-02 : Increase BasePool size to 50,000,000 SDR
    - 2022-05-11 : Increase BasePool  from 50M to 100M SDR [After crash]
    
- Basepool 7M → 13M SDT
PoolRecoveryPeriod 200 → 130 blocks

### Market Module min_spread parameter history
    - 2020/09/02 : changed from 2% to 0.5%


### Market Module pool_recovery_period parameter history
    - 2020-11-20 : Change PoolRecoveryPeriod from 24hr(14400) to 8hr(4800)
    - 2021-02-10 : PoolRecoveryPeriod 200 → 130 blocks
    - 2021-05-23 : Reduce PoolRecoveryPeriod to 49 blocks
    - 2022-02-02 : Reduce PoolRecoveryPeriod to 36 blocks
    - 2022-05-11 : Reduce PoolRecoveryPeriod to 18 blocks [After crash]




Mentioned by Jump - Split out TerraPoolDelta into TerraPoolDeltaBid and TerraPoolDeltaAsk.


In [72]:
# Initial Parameters
init_basepool = 50_000_000   # 50 M SDR
init_minstabilityspread = 0.05    # 5%
init_poolrecoveryperiod = 36      # regression parameter



In [79]:

def simulate_terra_classic(terra_classic_instance, market_swap_txs_DF, crash_analysis_DF, oracle_exchange_rate_DF, start_height, end_height):
    # DataFrame to store simulated swaps metrics
    simulation_metrics_DF = pd.DataFrame(columns=['BlockHeight','burnt_denom', 'tokens_burnt', 'minted_denom',
       'tokens_minted', 'swap_amount', 'swap_fee_amount','spread', 'terraPool','lunaPool', 'delta','ukrw_oracle_price','usdr_oracle_price'\
                                                 ,'uusd_oracle_price','usd_per_ust_price','usd_per_luna_price', 'expected_profit'])
    
    # DataFrame to store metrics around burning $100k worth UST for LUNA in each block
    simulated_ust_burn_metrics_DF = pd.DataFrame(columns=['BlockHeight', 'uusd_burnt', 'uluna_minted' ,'spread','delta', 'lunaPool','terra_pool','usd_per_luna_price', 'usd_per_ust_price'])

    # DataFrame to store metrics around burning $100k worth LUNA for UST in each block
    simulated_uluna_burn_metrics_DF = pd.DataFrame(columns=['BlockHeight', 'uluna_burnt','uusd_minted','spread','delta', 'lunaPool','terra_pool','usd_per_luna_price', 'usd_per_ust_price'])

    cur_height = start_height
    prices = {"ukrw": 0, "usdr": 0, "uusd": 0}    
    price_till = 0
    
    # Process all blocks 
    for cur_height in tqdm_notebook(range(start_height,end_height)):
        
        # Oracle Prices
        ukrw_oracle_price = terra_classic_instance.OracleKeeper.GetLunaExchangeRate("ukrw")
        usdr_oracle_price = terra_classic_instance.OracleKeeper.GetLunaExchangeRate("usdr")
        uusd_oracle_price = terra_classic_instance.OracleKeeper.GetLunaExchangeRate("uusd")
        
        # Get UST and LUNA's actual USD Price ()
        crash_analysis_DF_cur_block = crash_analysis_DF.loc[ crash_analysis_DF["BlockHeight"] == cur_height ]        
        ust_price_in_usd = crash_analysis_DF_cur_block.iloc[0]["ExchangeRate::USD/UST"]
        luna_price_in_usd = crash_analysis_DF_cur_block.iloc[0]["ExchangeRate::USD/LUNA"]
                            
        # Get all market swap txs to be processed for the current block
        cur_swap_txs_DF = market_swap_txs_DF.loc[ market_swap_txs_DF["BlockHeight"] == cur_height ]
        
        # Loop over all txs for each block
        for tx_row in cur_swap_txs_DF.to_dict(orient='records'):
            try:
                # Process swap tx :: Only UST burn txs are considered
                if tx_row["burnt_denom"] == "uusd" and tx_row["minted_denom"] == "uluna": 
                    # print(f"\nblock {cur_height} :: Burn {tx_row['burnt_denom']} {tx_row['tokens_burnt']} tokens and mint {tx_row['minted_denom']} tokens")
                    swap_response = terra_classic_instance.Swap("trader", {"denom":tx_row["burnt_denom"], "amount":float(tx_row["tokens_burnt"]) }, tx_row["minted_denom"])
                    swapCoin = swap_response["SwapCoin"] 
                    swapFee = swap_response["SwapFee"] 
                    spread = swap_response["spread"] 
                    err = swap_response["Error"] 

                    if err != None:
                        print(err)
                    else:
                        # Query market module params
                        terraPool = terra_classic_instance.get_TerraPool()
                        lunaPool = terra_classic_instance.get_LunaPool_fromTerraPool()
                        delta = terra_classic_instance.GetTerraPoolDelta()
                        
                        expected_profit = 0
                        if tx_row["burnt_denom"] == "uusd": # If UST is burnt and LUNA is minted
                            expected_profit = (float(swapCoin["amount"]) * luna_price_in_usd) -  (tx_row['tokens_burnt'] * ust_price_in_usd)
                        else:  # If LUNA is burnt and UST is minted
                            expected_profit = (float(swapCoin["amount"]) * ust_price_in_usd) -  (tx_row['tokens_burnt'] * luna_price_in_usd)
                        
                        try:
                            # store metrics
                            simulation_metrics_DF.loc[len(simulation_metrics_DF.index)] = [cur_height, tx_row["burnt_denom"],\
                                                        tx_row["tokens_burnt"], tx_row["minted_denom"], float(swapCoin["amount"]) + float(swapFee["amount"]),\
                                                        swapCoin["amount"], swapFee["amount"], spread, terraPool, lunaPool, delta, ukrw_oracle_price[0],\
                                                        usdr_oracle_price[0], uusd_oracle_price[0], ust_price_in_usd, luna_price_in_usd, expected_profit ]
#                             print(f"simulation_metrics_DF Length = {len(simulation_metrics_DF.index)}")
                        except Exception as e:    
                            print(e)
            except Exception as e:
                print(f"\nblock {cur_height} :: Burn {tx_row['burnt_denom']} {tx_row['tokens_burnt']} tokens and mint {tx_row['minted_denom']} tokens")
                print(e)
                
                
        # Query market module params
        terraPool = terra_classic_instance.get_TerraPool()
        lunaPool = terra_classic_instance.get_LunaPool_fromTerraPool()
        delta = terra_classic_instance.GetTerraPoolDelta()            
        
        # DataFrame to store metrics around burning $100k worth UST for LUNA in each block
        if float(ust_price_in_usd) > 0:
            ustc_tokens_to_burn = int(100_000 / float(ust_price_in_usd))
            ust_burn_res = terra_classic_instance.simulateSwap({"denom":"uusd","amount": ustc_tokens_to_burn }, "uluna")
            ust_burn_res_returnCoin = ust_burn_res["return_denom"] 
            ust_burn_res_spread = ust_burn_res["spread"] 
            ust_burn_res_err = ust_burn_res["error"]     
            if ust_burn_res_err!= None:
                print(f"UST Burn Simulation Error : {ust_burn_res_err}")
            else:
                simulated_ust_burn_metrics_DF.loc[len(simulated_ust_burn_metrics_DF.index)] =  [cur_height, ustc_tokens_to_burn, ust_burn_res_returnCoin["amount"],ust_burn_res_spread,delta, lunaPool, terraPool,luna_price_in_usd, ust_price_in_usd]

            
        # DataFrame to store metrics around burning $100k worth LUNA for UST in each block
        if float(luna_price_in_usd) > 0:
            uluna_tokens_to_burn = int(100_000 / float(luna_price_in_usd))
            uluna_burn_res = terra_classic_instance.simulateSwap({"denom":"uluna","amount": uluna_tokens_to_burn }, "uusd")
            uluna_burn_res_returnCoin = uluna_burn_res["return_denom"] 
            uluna_burn_res_spread = uluna_burn_res["spread"] 
            uluna_burn_res_err = uluna_burn_res["error"]     
            if uluna_burn_res_err!= None:
                print(f"ULUNA Burn Simulation Error : {uluna_burn_res_err}")
            else:
                simulated_uluna_burn_metrics_DF.loc[len(simulated_uluna_burn_metrics_DF.index)] =  [cur_height, uluna_tokens_to_burn, uluna_burn_res_returnCoin["amount"],uluna_burn_res_spread,delta, lunaPool, terraPool,luna_price_in_usd, ust_price_in_usd]

               
        # Replensih Market module's pool after each block
        terra_classic_instance.TerraMarketModuleEndBlock()
                            
                        
        # Update oracle prices every 5 blocks
        cur_price_DF = oracle_exchange_rate_DF.loc[ oracle_exchange_rate_DF["BlockHeight"] == cur_height ] 
    
        # Set oracle prices 
        terra_classic_instance.OracleKeeper.SetLunaExchangeRate("ukrw", cur_price_DF.iloc[0]["ukrw::mean"] )
#         print(f"{cur_height} || ukrw price = {cur_price_DF.iloc[0]['ukrw::mean']}")
        terra_classic_instance.OracleKeeper.SetLunaExchangeRate("usdr", cur_price_DF.iloc[0]["usdr::mean"] )
#         print(f"{cur_height} || usdr price = {cur_price_DF.iloc[0]['usdr::mean']}")
        terra_classic_instance.OracleKeeper.SetLunaExchangeRate("uusd", cur_price_DF.iloc[0]["uusd::mean"] )
#         print(f"{cur_height} || uusd price = {cur_price_DF.iloc[0]['uusd::mean']}")
                    
    
    return (simulation_metrics_DF, simulated_ust_burn_metrics_DF, simulated_uluna_burn_metrics_DF)






#### Create TerraMarketModule Instance 

In [80]:
# CREATE TERRA CLASSIC INSTANCE 
terra_classic_instance = TerraMarketModule(init_basepool, init_minstabilityspread, init_poolrecoveryperiod)
terra_classic_instance.BankKeeper.MintCoins("trader",{"denom":"uluna", "amount":800 * 10**6 })
terra_classic_instance.BankKeeper.MintCoins("trader",{"denom":"uusd", "amount":18000 * 10**6 })

# Set initial exchange rates
terra_classic_instance.OracleKeeper.SetLunaExchangeRate("uusd", 3.074 )
terra_classic_instance.OracleKeeper.SetLunaExchangeRate("usdr", 2.290 )
terra_classic_instance.OracleKeeper.SetLunaExchangeRate("ukrw", 3916.569 )



#### Execute simulation

In [82]:
# new_END_BLOCK = END_BLOCK - 10
# start_height = new_END_BLOCK - 100
start_height = START_BLOCK
end_height =  END_BLOCK

simul_name = "replicate_"
(simulation_metrics_DF, simulated_ust_burn_metrics_DF, simulated_uluna_burn_metrics_DF) =  simulate_terra_classic(terra_classic_instance, market_swap_txs_DF, crash_analysis_DF, aggregated_exchange_rate_vote_txs_DF, start_height, end_height)


  0%|          | 0/520059 [00:00<?, ?it/s]


block 7118009 :: Burn uusd 21000.0 tokens and mint uluna tokens
tuple indices must be integers or slices, not str


In [87]:
simulated_ust_burn_metrics_DF.head(3)

,BlockHeight,uusd_burnt,uluna_minted,spread,delta,lunaPool,terra_pool,usd_per_luna_price,usd_per_ust_price
0,"7,117,990.0000000","100,406.0000000",NaN,NaN,0.0000000,"50,000,000.0000000","50,000,000.0000000",110.7717770,0.9959540
1,"7,117,991.0000000","100,403.0000000",861.0753803,0.0500000,0.0000000,"50,000,000.0000000","50,000,000.0000000",110.7781370,0.9959793
2,"7,117,992.0000000","100,464.0000000",861.5490618,0.0500000,0.0000000,"50,000,000.0000000","50,000,000.0000000",110.7115950,0.9953810


In [88]:
# ------------------------------------------------------------
# START BLOCK : 2022.04.04 19:25:31 + 04:00   -   https://finder.terra.money/classic/blocks/7117994
start_block_time_swaps = datetime.datetime(2022, 4, 4, 19, 25, 31)
ttuple = start_block_time_swaps.timetuple()
start_block_timestamp = calendar.timegm(ttuple)

# Calculate timestamp for simulation_metrics_DF 
simulation_metrics_DF["timestamp"] = 0
simulation_metrics_DF.loc[0, 'timestamp'] = start_block_timestamp
for i in tqdm_notebook(range(1, len(simulation_metrics_DF))):
    new_timestamp = int(simulation_metrics_DF.loc[i-1, 'timestamp']) + ((int(simulation_metrics_DF.loc[i, 'BlockHeight']) - int(simulation_metrics_DF.loc[i-1, 'BlockHeight']))*average_block_time )
    new_timestamp = int(new_timestamp)
    simulation_metrics_DF.loc[i, 'timestamp'] = int(new_timestamp)

# Calculate dateTime for simulation_metrics_DF
simulation_metrics_DF["dateTime"] = simulation_metrics_DF.progress_apply(lambda x: datetime.datetime.fromtimestamp(int(x["timestamp"])) , axis=1)
# ------------------------------------------------------------    
# ------------------------------------------------------------
# START BLOCK : 2022.04.04 19:25:31 + 04:00   -   https://finder.terra.money/classic/blocks/7117990
start_block_time_swaps = datetime.datetime(2022, 4, 4, 19, 25, 4)
ttuple = start_block_time_swaps.timetuple()
start_block_timestamp = calendar.timegm(ttuple)

# Calculate timestamp for simulated_ust_burn_metrics_DF 
simulated_ust_burn_metrics_DF["timestamp"] = 0
simulated_ust_burn_metrics_DF.loc[0, 'timestamp'] = start_block_timestamp
for i in tqdm_notebook(range(1, len(simulated_ust_burn_metrics_DF))):
    new_timestamp = simulated_ust_burn_metrics_DF.loc[i-1, 'timestamp'] + ((int(simulated_ust_burn_metrics_DF.loc[i, 'BlockHeight']) - int(simulated_ust_burn_metrics_DF.loc[i-1, 'BlockHeight']))*average_block_time )
    new_timestamp = int(new_timestamp)
    simulated_ust_burn_metrics_DF.loc[i, 'timestamp'] = int(new_timestamp)
#     print(f"{simulated_ust_burn_metrics_DF.loc[i, 'BlockHeight']} -- {simulated_ust_burn_metrics_DF.loc[i, 'timestamp']}")

# Calculate dateTime for simulated_ust_burn_metrics_DF
simulated_ust_burn_metrics_DF["dateTime"] = simulated_ust_burn_metrics_DF.progress_apply(lambda x: datetime.datetime.fromtimestamp(int(x["timestamp"])) , axis=1)
# ------------------------------------------------------------
# ------------------------------------------------------------
# START BLOCK : 2022.04.04 19:25:31 + 04:00   -   https://finder.terra.money/classic/blocks/7117990
start_block_time_swaps = datetime.datetime(2022, 4, 4, 19, 25, 4)
ttuple = start_block_time_swaps.timetuple()
start_block_timestamp = calendar.timegm(ttuple)

# Calculate timestamp for simulated_uluna_burn_metrics_DF 
simulated_uluna_burn_metrics_DF["timestamp"] = 0
simulated_uluna_burn_metrics_DF.loc[0, 'timestamp'] = start_block_timestamp
for i in tqdm_notebook(range(1, len(simulated_uluna_burn_metrics_DF))):
    prev_timestamp = simulated_uluna_burn_metrics_DF.loc[i-1, 'timestamp']
    prev_blockheight = int(simulated_uluna_burn_metrics_DF.loc[i-1, 'BlockHeight'])
    cur_blockheight = int(simulated_uluna_burn_metrics_DF.loc[i, 'BlockHeight'])
                        
    new_timestamp = prev_timestamp + ((cur_blockheight - prev_blockheight)*average_block_time)
    new_timestamp = int(new_timestamp)
#     print(f"prev_blockheight={prev_blockheight} -- cur_blockheight = {cur_blockheight} || prev_timestamp={prev_timestamp} -- new_timestamp = {new_timestamp}")
                        
    simulated_uluna_burn_metrics_DF.loc[i, 'timestamp'] = int(new_timestamp)

# Calculate dateTime for simulated_uluna_burn_metrics_DF
simulated_uluna_burn_metrics_DF["dateTime"] = simulated_uluna_burn_metrics_DF.progress_apply(lambda x: datetime.datetime.fromtimestamp(int(x["timestamp"])) , axis=1)
           

  0%|          | 0/196335 [00:00<?, ?it/s]

100%|██████████| 196336/196336 [00:01<00:00, 183787.89it/s]


  0%|          | 0/520056 [00:00<?, ?it/s]

100%|██████████| 520057/520057 [00:02<00:00, 199994.84it/s]


  0%|          | 0/520056 [00:00<?, ?it/s]

100%|██████████| 520057/520057 [00:02<00:00, 201077.19it/s]


In [89]:
simulated_ust_burn_metrics_DF["BlockHeight"] = simulated_ust_burn_metrics_DF.progress_apply(lambda x : int(x["BlockHeight"]), axis=1)
simulated_uluna_burn_metrics_DF["BlockHeight"] = simulated_uluna_burn_metrics_DF.progress_apply(lambda x : int(x["BlockHeight"]), axis=1)



In [111]:
simulation_metrics_DF[simulation_metrics_DF["usd_per_luna_price"] > 1].tail(3)

,BlockHeight,burnt_denom,tokens_burnt,minted_denom,tokens_minted,swap_amount,swap_fee_amount,spread,terraPool,lunaPool,delta,ukrw_oracle_price,usdr_oracle_price,uusd_oracle_price,usd_per_ust_price,usd_per_luna_price,expected_profit,timestamp,dateTime
135380,7597331,uusd,"2,500.0000000",uluna,"2,553.1007345","2,339.6327191",213.4680154,0.0836113,"52,232,148.8342405","47,863,242.3860980","2,232,148.8342405","1,256.0541358",0.7307677,0.9792015,0.8828802,1.0259552,193.1578288,1652332792,2022-05-12 07:19:52
135381,7597331,uusd,"3,200.0000000",uluna,"3,267.9689402","2,994.4859971",273.4829431,0.0836859,"52,234,536.9604397","47,861,054.1124046","2,234,536.9604397","1,256.0541358",0.7307677,0.9792015,0.8828802,1.0259552,246.9918075,1652332792,2022-05-12 07:19:52
135382,7597331,uusd,"3,420.0000000",uluna,"3,492.6418048","3,200.0542287",292.5875761,0.0837726,"52,237,089.2703150","47,858,715.6160840","2,237,089.2703150","1,256.0541358",0.7307677,0.9792015,0.8828802,1.0259552,263.6619574,1652332792,2022-05-12 07:19:52


In [109]:
simulation_metrics_DF[simulation_metrics_DF["usd_per_luna_price"] < 1].head(3)

,BlockHeight,burnt_denom,tokens_burnt,minted_denom,tokens_minted,swap_amount,swap_fee_amount,spread,terraPool,lunaPool,delta,ukrw_oracle_price,usdr_oracle_price,uusd_oracle_price,usd_per_ust_price,usd_per_luna_price,expected_profit,timestamp,dateTime
104258,7590847,uusd,"90,000.0000000",uluna,"90,406.7154828","51,377.6081006","39,029.1073822",0.4317058,"66,359,369.9274174","37,673,654.8694548","16,359,369.9274174","1,406.7600819",0.7393742,0.9955013,0.4613949,0.9486852,"7,215.6313535",1652293082,2022-05-11 20:18:02
104259,7590847,uusd,"1,087,710.8458660",uluna,"1,092,626.2774418","612,846.8529758","479,779.4244660",0.4391066,"67,167,229.5904974","37,220,531.7271816","17,167,229.5904974","1,406.7600819",0.7393742,0.9955013,0.4613949,0.9486852,"79,534.4533816",1652293082,2022-05-11 20:18:02
104260,7590847,uusd,"10,256.3988240",uluna,"10,302.7481151","5,708.5915222","4,594.1565929",0.4459156,"67,174,847.1764853","37,216,310.9419790","17,174,847.1764853","1,406.7600819",0.7393742,0.9955013,0.4613949,0.9486852,683.4057200,1652293082,2022-05-11 20:18:02


In [90]:
simulation_metrics_DF["tokens_minted"].sum()

2714272160354.5317

In [107]:
simulation_metrics_DF.head(10)

,BlockHeight,burnt_denom,tokens_burnt,minted_denom,tokens_minted,swap_amount,swap_fee_amount,spread,terraPool,lunaPool,delta,ukrw_oracle_price,usdr_oracle_price,uusd_oracle_price,usd_per_ust_price,usd_per_luna_price,expected_profit,timestamp,dateTime
0,7117994,uusd,"35,000.0000000",uluna,316.1367154,300.3298796,15.8068358,0.0500000,"50,025,325.0691932","49,974,687.7514957","25,325.0691932","134,588.0821005",80.1079658,110.7115950,1.0014010,110.7115950,"-1,799.0363222",1649100331,2022-04-04 21:25:31
1,7117996,uusd,199.4200000,uluna,1.8012567,1.7111938,0.0900628,0.0500000,"50,024,081.9568617","49,975,929.6363675","24,081.9568617","134,588.0821005",80.1079658,110.7115950,1.0026252,110.7661450,-10.4011767,1649100344,2022-04-04 21:25:44
2,7118012,uusd,3.7358990,uluna,0.0337660,0.0320777,0.0016883,0.0500000,"50,029,310.4155546","49,970,706.7563882","29,310.4155546","132,790.0000000",80.1360382,110.6409710,1.0020857,110.6357886,-0.1947531,1649100453,2022-04-04 21:27:33
3,7118013,uusd,51.7316180,uluna,0.4675848,0.4442056,0.0233792,0.0500000,"50,028,533.6686965","49,971,482.6054213","28,533.6686965","134,495.5527469",80.0525341,110.6357886,1.0028985,110.6357886,-2.7365246,1649100459,2022-04-04 21:27:39
4,7118019,uusd,9.3973470,uluna,0.0848650,0.0806217,0.0042432,0.0500000,"50,024,103.1281462","49,975,908.4854710","24,103.1281462","134,614.5537853",80.1226844,110.7329182,0.9978686,110.7329182,-0.4498376,1649100499,2022-04-04 21:28:19
5,7118291,uusd,3.5972720,uluna,0.0325345,0.0309078,0.0016267,0.0500000,"50,000,013.9347532","49,999,986.0652506",13.9347532,"134,359.0376144",80.0039702,110.5678182,1.0005901,110.5678182,-0.1819862,1649102354,2022-04-04 21:59:14
6,7118322,uusd,"10,000.0000000",uluna,90.8175683,86.2766899,4.5408784,0.0500000,"50,007,248.7087692","49,992,752.3419540","7,248.7087692","132,398.0000000",79.7520803,110.1108540,1.0024314,110.4150727,-498.0675044,1649102565,2022-04-04 22:02:45
7,7118324,uusd,"1,009.0000000",uluna,9.1382451,8.6813329,0.4569123,0.0500000,"50,007,581.6815285","49,992,419.4679351","7,581.6815285","134,173.3327223",79.8934207,110.4150727,1.0024314,110.4150727,-52.9033316,1649102578,2022-04-04 22:02:58
8,7118325,uusd,"11,092.0000000",uluna,100.4572993,95.4344343,5.0228650,0.0500000,"50,015,396.9565375","49,984,607.7833284","15,396.9565375","134,173.3327223",79.8934207,110.4150727,1.0024314,110.4150727,-581.5696277,1649102584,2022-04-04 22:03:04
9,7118333,uusd,61.6483770,uluna,0.5561511,0.5283435,0.0278076,0.0500000,"50,012,334.8151070","49,987,668.2270958","12,334.8151069","134,697.5008910",80.2068357,110.8482518,1.0063642,110.8482518,-3.4747599,1649102638,2022-04-04 22:03:58


In [108]:
simulated_ust_burn_metrics_DF.head(10)

,BlockHeight,uusd_burnt,uluna_minted,spread,delta,lunaPool,terra_pool,usd_per_luna_price,usd_per_ust_price,timestamp,dateTime
0,7117990,"100,406.0000000",NaN,NaN,0.0000000,"50,000,000.0000000","50,000,000.0000000",110.7717770,0.9959540,1649100304,2022-04-04 21:25:04
1,7117991,"100,403.0000000",861.0753803,0.0500000,0.0000000,"50,000,000.0000000","50,000,000.0000000",110.7781370,0.9959793,1649100310,2022-04-04 21:25:10
2,7117992,"100,464.0000000",861.5490618,0.0500000,0.0000000,"50,000,000.0000000","50,000,000.0000000",110.7115950,0.9953810,1649100316,2022-04-04 21:25:16
3,7117993,"100,463.0000000",862.0583057,0.0500000,0.0000000,"50,000,000.0000000","50,000,000.0000000",110.7115950,0.9953834,1649100322,2022-04-04 21:25:22
4,7117994,"99,860.0000000",856.8840509,0.0500000,"25,325.0691932","49,974,687.7514957","50,025,325.0691932",110.7115950,1.0014010,1649100328,2022-04-04 21:25:28
5,7117995,"99,795.0000000",856.3262954,0.0500000,"24,621.5950490","49,975,390.5234424","50,024,621.5950490",110.7115950,1.0020515,1649100334,2022-04-04 21:25:34
6,7117996,"99,738.0000000",855.8371867,0.0500000,"24,081.9568617","49,975,929.6363675","50,024,081.9568617",110.7661450,1.0026252,1649100340,2022-04-04 21:25:40
7,7117997,"99,891.0000000",856.7279289,0.0500000,"23,413.0136155","49,976,597.9446373","50,023,413.0136155",110.5953926,1.0010892,1649100346,2022-04-04 21:25:46
8,7117998,"99,891.0000000",858.0506636,0.0500000,"22,762.6521262","49,977,247.7059249","50,022,762.6521262",110.5953926,1.0010892,1649100352,2022-04-04 21:25:52
9,7117999,"99,891.0000000",858.0506636,0.0500000,"22,130.3562338","49,977,879.4344861","50,022,130.3562338",110.5953926,1.0010892,1649100358,2022-04-04 21:25:58


In [112]:
# SAVE Files 
folder_name = "./simulated_datasets/crash_analysis/"
simul_name = "crash_"
simulation_metrics_DF.to_csv(folder_name + simul_name + "simulation_metrics_DF.csv")
simulated_ust_burn_metrics_DF.to_csv(folder_name + simul_name + "simulated_ust_burn_metrics_DF.csv")
simulated_uluna_burn_metrics_DF.to_csv(folder_name + simul_name + "simulated_uluna_burn_metrics_DF.csv")

In [113]:
simulation_metrics_DF.tail(100)

,BlockHeight,burnt_denom,tokens_burnt,minted_denom,tokens_minted,swap_amount,swap_fee_amount,spread,terraPool,lunaPool,delta,ukrw_oracle_price,usdr_oracle_price,uusd_oracle_price,usd_per_ust_price,usd_per_luna_price,expected_profit,timestamp,dateTime
196236,7607775,uusd,10.0000000,uluna,"3,433.7262840","3,262.0399698",171.6863142,0.0500000,"51,080,168.0645944","48,942,673.7366756","1,080,168.0645944",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-19,423.9690191",1652397530,2022-05-13 01:18:50
196237,7607775,uusd,26.0000000,uluna,"8,927.6883383","8,481.3039214",446.3844169,0.0500000,"51,080,187.4661694","48,942,655.1469836","1,080,187.4661694",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-50,502.3194498",1652397530,2022-05-13 01:18:50
196238,7607775,uusd,1.0000000,uluna,343.3726284,326.2039970,17.1686314,0.0500000,"51,080,188.2123838","48,942,654.4319957","1,080,188.2123838",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-1,942.3969019",1652397530,2022-05-13 01:18:50
196239,7607775,uusd,1.2574380,uluna,431.7697911,410.1813016,21.5884896,0.0500000,"51,080,189.1507021","48,942,653.5329428","1,080,189.1507021",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-2,442.4436755",1652397530,2022-05-13 01:18:50
196240,7607775,uusd,5.0000000,uluna,"1,716.8631420","1,631.0199849",85.8431571,0.0500000,"51,080,192.8817743","48,942,649.9580039","1,080,192.8817743",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-9,711.9845096",1652397530,2022-05-13 01:18:50
196241,7607775,uusd,0.5000000,uluna,171.6863142,163.1019985,8.5843157,0.0500000,"51,080,193.2548815","48,942,649.6005100","1,080,193.2548815",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,-971.1984510,1652397530,2022-05-13 01:18:50
196242,7607775,uusd,400.0000000,uluna,"137,349.0513589","130,481.5987910","6,867.4525679",0.0500000,"51,080,491.7406500","48,942,363.6070929","1,080,491.7406500",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-776,958.7607657",1652397530,2022-05-13 01:18:50
196243,7607775,uusd,100.0000000,uluna,"34,337.2628397","32,620.3996977","1,716.8631420",0.0500000,"51,080,566.3620921","48,942,292.1092609","1,080,566.3620921",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-194,239.6901914",1652397530,2022-05-13 01:18:50
196244,7607775,uusd,5.0000000,uluna,"1,716.8631420","1,631.0199849",85.8431571,0.0500000,"51,080,570.0931642","48,942,288.5343748","1,080,570.0931642",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-9,711.9845096",1652397530,2022-05-13 01:18:50
196245,7607775,uusd,2.0000000,uluna,686.7452568,652.4079940,34.3372628,0.0500000,"51,080,571.5855931","48,942,287.1044205","1,080,571.5855931",3.7518569,0.0021732,0.0029123,"1,943.3469019",0.0029123,"-3,884.7938038",1652397530,2022-05-13 01:18:50


In [ ]:
type(simulated_ust_burn_metrics_DF.loc[0,"BlockHeight"])

In [33]:


simulation_metrics_DF = pd.read_csv("crash_simulation_metrics_DF.csv")






![title](./terra_classic_dataset/1.png)

![title](./terra_classic_dataset/2.png)

![title](./terra_classic_dataset/3.png)

![title](./terra_classic_dataset/4.png)




# TERRA Market Module

A simulated version of terra's market module. Used to simulate on-chain behaviour for analysis. 


### Parameters

The market module contains the following parameters:

| Key                 | Type         | Example                |
|---------------------|--------------|------------------------|
| basepool            | string (dec) | "250000000000.0"       |
| minstabilityspread  | string (dec) | "0.010000000000000000"                                           |
| poolrecoveryperiod  | string (int) | "14400"                |


-------------------------x-------------------------x-------------------------x-------------------------x--------------

## State

### TerraPoolDelta

Market module provides swap functionality based on constant product mechanism. Terra pool have to keep its delta to track the currency demands for swap spread. Luna pool can be retrived from Terra pool delta with following equation:

```go
TerraPool = BasePool + delta
LunaPool = (BasePool * BasePool) / TerraPool
```

> Note that the all pool holds decimal unit of `usdr` amount, so delta is also `usdr` unit.


### Messages

#### MsgSwap

A MsgSwap transaction denotes the Trader's intent to swap their balance of `OfferCoin` for new denomination `AskDenom`, for both Terra<>Terra and Terra<>Luna swaps.


#### MsgSwapSend
A MsgSendSwap first performs a swap of OfferCoin into AskDenom and the sends the resulting coins to ToAddress. Tax is charged normally, as if the sender were issuing a MsgSend with the resutling coins of the swap.


-------------------------x-------------------------x-------------------------x-------------------------x--------------

### Functions

#### ComputeSwap

This function detects the swap type from the offer and ask denominations and returns:

1. The amount of asked coins that should be returned for a given `offerCoin`. This is achieved by first spot-converting `offerCoin` to µSDR and then from µSDR to the desired `askDenom` with the proper exchange rate reported from by the Oracle.

2. The spread % that should be taken as a swap fee given the swap type. Terra<>Terra swaps simply have the Tobin Tax spread fee. Terra<>Luna spreads are the greater of `MinSpread` and spread from Constant Product pricing.


#### ApplySwapToPool

1. This function is called during the swap to update the blockchain's measure of , `TerraPoolDelta`, when the balances of the Terra and Luna liquidity pools have changed.

2. Terra currencies share the same liquidity pool, so `TerraPoolDelta` remains unaltered during Terra<>Terra swaps.

3. For Terra<>Luna swaps, the relative sizes of the pools will be different after the swap, and `delta` will be updated with the following formulas:

    - For Terra to Luna, `delta = delta + offerAmount`
    - For Luna to Terra, `delta = delta - askAmount`


-------------------------x-------------------------x-------------------------x-------------------------x--------------

### End Block

#### Replenish Pool
At each `EndBlock`, the value of `TerraPoolDelta` is decreased depending on `PoolRecoveryPeriod` of parameter.

This allows the network to sharply increase spread fees in during acute price fluctuations, and automatically return the spread to normal after some time when the price change is long term.




In [1]:
# Market Module implements the logic of swaps between Luna and Terra coins implemented via burn / mint 
# making their total supply dynamic

# Market module provides swap functionality based on constant product mechanism. Terra pool have to keep 
# its delta to track the currency demands for swap spread. 

#---------------x-----------x--------------
class TerraMarketModule:
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)    

    def __init__(self, basepool, minstabilityspread, poolrecoveryperiod):
        self.delta = 0
        self.SetParams(basepool, minstabilityspread, poolrecoveryperiod)
    
        self.MicroLunaDenom = "uluna" 
        self.MicroUSDDenom  = "uusd"
        self.MicroKRWDenom  = "ukrw"
        self.MicroSDRDenom  = "usdr"
        self.MicroCNYDenom  = "ucny"
        self.MicroJPYDenom  = "ujpy"
        self.MicroEURDenom  = "ueur"
        self.MicroGBPDenom  = "ugbp"
        self.MicroMNTDenom  = "umnt"
    
        # Keeps track of token balances
        self.BankKeeper = TerraBankModule()
        # Keeps track of oracle prices
        self.OracleKeeper = TerraOracleKeeper()
   
    # SET Parameters
    def SetParams(self, basepool, minstabilityspread, poolrecoveryperiod):
        self.basepool = basepool
        self.minstabilityspread = minstabilityspread
        self.poolrecoveryperiod = poolrecoveryperiod

     # Terra pool value is returned with following function:
    def get_TerraPool(self):
        terraPool = self.basepool + self.delta
        return terraPool
        
        
    # Luna pool can be retrived from Terra pool delta with following function:
    def get_LunaPool_fromTerraPool(self):
        terraPool = self.basepool + self.delta
        lunaPool = (self.basepool * self.basepool) / terraPool
        return lunaPool

    # GetBasePool returns the basepool
    def GetBasePool(self):
        return self.basepool
    
    # GetMinStabilitySpread returns the minstabilityspread
    def GetMinStabilitySpread(self):
        return self.minstabilityspread

    # GetPoolRecoveryPeriod returns the poolrecoveryperiod
    def GetPoolRecoveryPeriod(self):
        return self.poolrecoveryperiod
    
    # GetTerraPoolDelta returns the gap between the TerraPool and the TerraBasePool
    def GetTerraPoolDelta(self):
        return self.delta

    # SetTerraPoolDelta updates TerraPoolDelta which is gap between the TerraPool and the BasePool
    def _SetTerraPoolDelta(self,delta):
        self.delta = delta

    # USER INTERACTION - SWAP FUNCTION
    def Swap(self, trader,offerCoin, askDenom ) : 
        res = self._handleSwapRequest(trader,offerCoin, askDenom )
        return res        


    # USER INTERACTION - SWAPSEND FUNCTION
    def SwapSend(self, trader, offerCoin, askDenom ): 
        res = self._handleSwapRequest(trader,offerCoin, askDenom )
        return res
        
    # INTERNAL FUNCTION
    # // handleMsgSwap handles the logic of a MsgSwap
    # // This function does not repeat checks that have already been performed
    # // Ex) assert(offerCoin.Denom != askDenom)
    def _handleSwapRequest(self, trader,offerCoin, askDenom ): 

        #  Compute exchange rates between the ask and offer
        (swapDecCoin, spread, err) = self._ComputeSwap(offerCoin, askDenom)
        if err != None:
            return (None, err)

        #  Charge a spread if applicable; the spread is burned
        feeDecCoin : Coin
        if spread > 0:
            feeDecCoin = {"denom" : swapDecCoin["denom"], "amount" : spread * swapDecCoin["amount"]}
        else:
            feeDecCoin = {"denom" : swapDecCoin["denom"], amount : 0 }
        
        # Subtract fee from the swap coin
        swapDecCoin["amount"] = swapDecCoin["amount"] - feeDecCoin["amount"]

        # Update pool delta
        err = self._ApplySwapToPool(offerCoin, swapDecCoin)
        if err != None:
            return (None, err)

        # Send offer coins to module account
        err = self.BankKeeper.SendCoinsFromAccountToModule(trader, "market", offerCoin)
        if err != None:
            return None, err

        # Burn offered coins and subtract from the trader's account
        err = self.BankKeeper.BurnCoins("market", offerCoin)
        if err != None:
            return None, err

        # Mint asked coins and credit Trader's account
        #         swapCoin, decimalCoin = swapDecCoin.TruncateDecimal()
        #         feeDecCoin = feeDecCoin.Add(decimalCoin) # add truncated decimalCoin to swapFee
        #         feeCoin, _ := feeDecCoin.TruncateDecimal()
        mintCoins = { "denom": swapDecCoin["denom"], "amount":swapDecCoin["amount"] + feeDecCoin["amount"] }
        err = self.BankKeeper.MintCoins("market", mintCoins)
        if err != None:
            return None, err

        # Send swap coin to the trader
        err = self.BankKeeper.SendCoinsFromModuleToAccount("market", trader, swapDecCoin)
        if err != None:
            return None, err

        # Send swap fee to oracle account
        if feeDecCoin["amount"] > 0:
            err = self.BankKeeper.SendCoinsFromModuleToModule("market", "oracle", feeDecCoin)
            if err != None:
                return None, err


        return { "SwapCoin": swapDecCoin,
                  "SwapFee":  feeDecCoin,
                    "spread": spread,
                    "Error": None
                }


    
    # INTERNAL FUNCTION
    # // ComputeSwap returns the amount of asked coins should be returned for a given offerCoin at the effective
    # // exchange rate registered with the oracle.
    # // Returns an Error if the swap is recursive, or the coins to be traded are unknown by the oracle, or the amount
    # // to trade is too small.
    def _ComputeSwap(self, offerCoin, askDenom):

        # Return invalid recursive swap err
        if offerCoin["denom"] == askDenom:
            return {"denom":"", "amount":0}, 0, "_ComputeSwap :: offer asset cannot be same as ask asset"

        # Swap offer coin to base denom (usdr) for simplicity of swap process
        baseOfferDecCoin, err = self._ComputeInternalSwap(offerCoin, self.MicroSDRDenom)
        if err != None:
            return {"denom":"", "amount":0}, 0, err
#         else:
#             print(f"{offerCoin['amount']} {offerCoin['denom']} --> {baseOfferDecCoin['amount']} {baseOfferDecCoin['denom']}")

        # Get Ask asset swap amount based on the oracle price
        retDecCoin, err = self._ComputeInternalSwap(baseOfferDecCoin, askDenom)
        if err != None:
            return {"denom":"", "amount":0}, 0, err
#         else:
#             print(f"{baseOfferDecCoin['amount']} {baseOfferDecCoin['denom']} --> {retDecCoin['amount']} {retDecCoin['denom']}")

        # Terra => Terra swap
        # Apply only tobin tax without constant product spread
        if offerCoin["denom"] != self.MicroLunaDenom and askDenom != self.MicroLunaDenom:
            # OfferCoin Toblin Tax
            offerTobinTax, err2 = self.OracleKeeper.GetTobinTax(offerCoin["denom"])
            if err2 != None :
                return {"denom":"", "amount":0}, 0, err2

            # AskAsset Toblin Tax
            askTobinTax, err2 = self.OracleKeeper.GetTobinTax(ctx, askDenom)
            if err2 != None :
                return {"denom":"", "amount":0}, 0, err2

            # Apply highest tobin tax for the denoms in the swap operation
            tobinTax = 0
            if askTobinTax > offerTobinTax:
                tobinTax = askTobinTax
            else :
                tobinTax = offerTobinTax
            
            # Return the computed returnAsset and spread for Terra --> Terra Swap
            spread = tobinTax
            return (retDecCoin, spread, None)

        basePool = self.basepool
        minSpread = self.minstabilityspread

        #  constant-product, which by construction is square of base(equilibrium) pool
        # Calculate current TerraPool and LunaPool values
        cp = basePool*basePool
        terraPoolDelta = self.GetTerraPoolDelta()
        terraPool = basePool + terraPoolDelta
        lunaPool = cp / terraPool
    
#         print(f"terraPool = {terraPool} lunaPool = {lunaPool} terraPoolDelta = {terraPoolDelta}")
    
        # Assign TerraPool / LunaPool to OfferAsset / AskAssets
        offerPool = "" # base denom(usdr) unit
        askPool = ""   # base denom(usdr) unit
        #  Terra->Luna swap
        if offerCoin["denom"] != self.MicroLunaDenom:
            offerPool = terraPool
            askPool = lunaPool
        # Luna->Terra swap
        else:
            offerPool = lunaPool
            askPool = terraPool

        # Get cp(constant-product) based swap amount
        # baseAskAmt = askPool - cp / (offerPool + offerBaseAmount)
        # baseAskAmt is base denom(usdr) unit
        exp_new_ask_pool = cp/(offerPool + baseOfferDecCoin["amount"])
        baseAskAmt = askPool - exp_new_ask_pool
#         print(f"askPool = {askPool} | exp_new_ask_pool = {exp_new_ask_pool} | baseAskAmt = {baseAskAmt} ")

        # Both baseOffer and baseAsk are usdr units, so spread can be calculated by
        # spread = (baseOfferAmt - baseAskAmt) / baseOfferAmt
        baseOfferAmount = baseOfferDecCoin["amount"]
        spread = (baseOfferAmount - baseAskAmt) / baseOfferAmount

        if spread < minSpread:
            spread = minSpread
#         print(f"spread = {spread} ")

        return(retDecCoin, spread, None)



    
    # INTERNAL FUNCTION
    # ComputeInternalSwap returns the amount of asked DecCoin should be returned for a given offerCoin at the effective
    # exchange rate registered with the oracle.
    # Different from ComputeSwap, ComputeInternalSwap does not charge a spread as its use is system internal.
    def _ComputeInternalSwap(self, offerCoin, askDenom):
        if offerCoin["denom"] == askDenom:
            return offerCoin, 0
        
        # Get exchange rate :: OfferAsset --> Luna
        offerRate, err = self.OracleKeeper.GetLunaExchangeRate(offerCoin["denom"])
        if err != None:
            return {"denom":"", "amount":0}, f"ErrNoEffectivePriceFromOracleFor ${offerCoin['denom']}"
        
        # Get exchange rate :: AskAsset --> Luna
        askRate, err = self.OracleKeeper.GetLunaExchangeRate(askDenom)
        if err != None:
            return {"denom":"", "amount":0}, f"ErrNoEffectivePriceFromOracleFor ${askDenom}"
        
        # Calculate return amount
        retAmount = offerCoin["amount"] * askRate / offerRate
        if retAmount < 0:
            return {"denom":"", "amount":0}, "ComputeInternalSwap::Err Return Calc"
        
        # return calc. return amount
        return ({"denom":askDenom,"amount": retAmount}, None)
    
    
    # INTERNAL FUNCTION
    # ApplySwapToPool updates each pool with offerCoin and askCoin taken from swap operation,
    # OfferPool = OfferPool + offerAmt (Fills the swap pool with offerAmt)
    # AskPool = AskPool - askAmt       (Uses askAmt from the swap pool)
    def  _ApplySwapToPool(self, offerCoin, askCoin):
        # No delta update in case Terra to Terra swap
        if offerCoin["denom"] != self.MicroLunaDenom and askCoin["denom"] != self.MicroLunaDenom:
            return None
        # Get Delta
        terraPoolDelta = self.GetTerraPoolDelta()

        # In case swapping Terra to Luna, the terra swap pool(offer) must be 
        # increased and the luna swap pool(ask) must be decreased
        if offerCoin["denom"] != self.MicroLunaDenom and askCoin["denom"] == self.MicroLunaDenom:
            offerBaseCoin, err = self._ComputeInternalSwap(offerCoin, self.MicroSDRDenom)
            if err != None:
                return err
            terraPoolDelta = terraPoolDelta + offerBaseCoin["amount"]

        # In case swapping Luna to Terra, the luna swap pool(offer) must be increased and the terra swap pool(ask) must be decreased
        if offerCoin["denom"] == self.MicroLunaDenom and askCoin["denom"] != self.MicroLunaDenom:
            askBaseCoin, err = self._ComputeInternalSwap(askCoin, self.MicroSDRDenom)
            if err != None:
                return err
            terraPoolDelta = terraPoolDelta - askBaseCoin["amount"]
        
        # Update Terra Delta Variable
        self._SetTerraPoolDelta(terraPoolDelta)

        return None
      
    # EXTERNAL QUERY FUNCTION : DOESN'T IMPACT STATE
    # simulateSwap interface for simulate swap
    def simulateSwap(self, offerCoin, askDenom):
        if askDenom == offerCoin["denom"] :
            return {"denom":"", "amount":0}, "askDenom and offerDenom cannot be same"

        # Invalid amount
#         if int(offerCoin["amount"]) > 100 :
#             return { 
#                 "return_denom": {"denom":"", "amount":0}, 
#                 "spread": 0,
#                 "error": "Invalid offerCoin"
#             }
        
        # Calculate Swap
        swapCoin, spread, err = self._ComputeSwap(offerCoin, askDenom)
        if err != None:
            return { 
                "return_denom": {"denom":"", "amount":0}, 
                "spread": 0,
                "error": err
            }
        
        # Subtract spread
        if spread > 0:
            swapFeeAmt = spread * swapCoin["amount"]
            if swapFeeAmt > 0:
                swapFee = { "denom":swapCoin["denom"], "amount":swapFeeAmt }
                swapCoin["amount"] = swapCoin["amount"] -  swapFee["amount"]

        return {
            "return_denom": swapCoin,
            "spread": spread,
            "error": None
        }
    

    # END--BLOCK FUNCTION    
    # ==> EndBlocker is called at the end of every block
    def TerraMarketModuleEndBlock(self):
        # Replenishes each pools towards equilibrium
        self._ReplenishPools()
    
    # INTERNAL FUNCTION
    # Replenishes each pools towards equilibrium
    def _ReplenishPools(self):
        # Get current pool delta
        poolDelta = self.GetTerraPoolDelta()
        # Calculate Pool Regression Amount with current Pool Recovery period
        poolRegressionAmt = poolDelta / self.poolrecoveryperiod
        # Replenish terra pool towards base pool. 
        # regressionAmt cannot make delta zero
        newPoolDelta = poolDelta - poolRegressionAmt
        # Update Delta
        self._SetTerraPoolDelta(newPoolDelta)
#         print(f"Replenishing pools : poolDelta = {poolDelta} | poolRegressionAmt = {poolRegressionAmt} | newPoolDelta = {newPoolDelta}")

        
        


 



# Bank Module 

Bank Module keeps track of user's token balances and the total supply of these tokens.

In [2]:
# BankKeeper defines expected supply keeper
#---------------x-----------x--------------
class TerraBankModule:
    
    
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
    
    def __init__(self):
        self.accounts = pd.DataFrame(columns=["user_address","uluna","uusd","usdr"])
        self.accounts.loc[0] = ["",0,0,0]
        self.totalSupply = {
        "uluna": 0,
        "uusd": 0,
        "usdr": 0,
    }
        
    # TRANSFER TOKENS
    def SendCoinsFromModuleToModule(self, senderAddr, recipientAddr,  coin):
        sender = self.accounts.loc[self.accounts['user_address'] == senderAddr]
        if len(sender) == 0: # Return Error 
            return "Sender doesn't exist"

        # balance check
        sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        if sender_balance < float(coin["amount"]):
            return "insufficient balance"        
        
        # Get recepient
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        if len(recepient) == 0: # Create user 
            self.accounts.loc[len(self.accounts.index)] = [recipientAddr,0,0,0]  
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        
        # Update recepient user balance in the dataframe
        recepient_balance = float(recepient[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == recipientAddr, coin["denom"]] = recepient_balance + float(coin["amount"])     
        # Update sender user balance in the dataframe
        self.accounts.loc[self.accounts['user_address'] == senderAddr, coin["denom"]] = sender_balance - float(coin["amount"])     
        
        # LOgging
        new_sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        new_recepient_balance = float(self.accounts.loc[self.accounts['user_address'] == recipientAddr][coin["denom"]])        
#         print(f"{float(coin['amount'])} {coin['denom']} TRANSFERRED FROM {senderAddr} TO {recipientAddr}")
        return None
    
    
    # TRANSFER TOKENS
    def SendCoinsFromAccountToModule(self, senderAddr, recipientAddr,  coin):
        sender = self.accounts.loc[self.accounts['user_address'] == senderAddr]
        if len(sender) == 0: # Return Error 
            return "Sender doesn't exist"

        # balance check
        sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        if sender_balance < float(coin["amount"]):
            return "insufficient balance"        
        
        # Get recepient
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        if len(recepient) == 0: # Create user 
            self.accounts.loc[len(self.accounts.index)] = [recipientAddr,0,0,0]  
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        
        # Update recepient user balance in the dataframe
        recepient_balance = float(recepient[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == recipientAddr, coin["denom"]] = recepient_balance + float(coin["amount"])     
        # Update sender user balance in the dataframe
        self.accounts.loc[self.accounts['user_address'] == senderAddr, coin["denom"]] = sender_balance - float(coin["amount"])     
        
        # LOgging
        new_sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        new_recepient_balance = float(self.accounts.loc[self.accounts['user_address'] == recipientAddr][coin["denom"]])        
#         print(f"{int(coin['amount'])} {coin['denom']} TRANSFERRED FROM {senderAddr} TO {recipientAddr}")
        return None
    
    
    # TRANSFER TOKENS
    def SendCoinsFromModuleToAccount(self, senderAddr, recipientAddr,  coin):
        sender = self.accounts.loc[self.accounts['user_address'] == senderAddr]
#         print(sender)
        if len(sender) == 0: # Return Error 
            return "Sender doesn't exist"

        # balance check
        sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        if sender_balance < float(coin["amount"]):
            return "insufficient balance"        
        
        # Get recepient
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
#         print(recepient)
        if len(recepient) == 0: # Create user 
            self.accounts.loc[len(self.accounts.index)] = [recipientAddr,0,0,0]  
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        
        # Update recepient user balance in the dataframe
        recepient_balance = float(recepient[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == recipientAddr, coin["denom"]] = recepient_balance + float(coin["amount"])     
        # Update sender user balance in the dataframe
        self.accounts.loc[self.accounts['user_address'] == senderAddr, coin["denom"]] = sender_balance - float(coin["amount"])     
        
        # LOgging
        new_sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        new_recepient_balance = float(self.accounts.loc[self.accounts['user_address'] == recipientAddr][coin["denom"]])        
#         print(f"{int(coin['amount'])} {coin['denom']} TRANSFERRED FROM {senderAddr} TO {recipientAddr}")
        return None    
    
    
    # MINT NEW TOKENS
    def MintCoins(self, recipientAddr, coin):
        # get recepient
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]
        if len(recepient) == 0: # Create user 
            self.accounts.loc[len(self.accounts.index)] = [recipientAddr,0,0,0]  
        recepient = self.accounts.loc[self.accounts['user_address'] == recipientAddr]

        # Update recepient user balance in the dataframe
        cur_balance = float(recepient[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == recipientAddr, coin["denom"]] = cur_balance + float(coin["amount"])     
        
        # Update total accounted token supply
        self.totalSupply[coin["denom"]] = self.totalSupply[coin["denom"]] + coin["amount"]

        # LOgging
        new_balance = float(self.accounts.loc[self.accounts['user_address'] == recipientAddr][coin["denom"]])
#         print(f"{int(coin['amount'])} {coin['denom']} MINTED")
        return None

       
    # BURN TOKENS
    def BurnCoins(self, senderAddr, coin):
        # get sender
        sender = self.accounts.loc[self.accounts['user_address'] == senderAddr]
        if len(sender) == 0: # Return Error 
            return "Sender doesn't exist"
        # balance check
        sender_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])        
        if sender_balance < float(coin["amount"]):
            return "insufficient balance"                
        
        # Update sender user balance in the dataframe
        cur_balance = float(sender[coin['denom']])
        self.accounts.loc[self.accounts['user_address'] == senderAddr, coin["denom"]] = cur_balance - float(coin["amount"])     
        
        # Update total accounted token supply
        self.totalSupply[coin["denom"]] = self.totalSupply[coin["denom"]] - coin["amount"]

        # LOgging
        new_balance = float(self.accounts.loc[self.accounts['user_address'] == senderAddr][coin["denom"]])
#         print(f"{float(coin['amount'])} {coin['denom']} BURNT")
        return None

           
        
    # GET USER BALANCE
    def GetBalance(self, addr, denom):
        user = self.accounts.loc[self.accounts['user_address'] == addr]
        len_ = len(user)
        if len(user) == 0:
            self.accounts.loc[len(self.accounts.index)] = [addr,0,0,0]
            return 0
        return float(user[denom])
        

    def GetTotalSupply(self, denom):
        return self.totalSupply[denom]
    
    
    def SpendableCoins(self,ctx, addr):
        pass

    def IsSendEnabledCoin(self,ctx, coin):
        pass

## ORACLE MODULE

A toned-down version of terra's Oracle module used to update / fetch prices for terraCoins to be used by Terra's market module for processing swaps.



In [3]:
# OracleKeeper defines expected oracle keeper
#---------------x-----------x--------------
class TerraOracleKeeper:

    def __new__(cls, *args, **kwargs):
        return super().__new__(cls)
    
    def __init__(self):    
        self.voteperiod = 30                       # core.BlocksPerMinute / 2 (30 seconds) 
        self.rewarddistributionwindow = 5256000    # core.BlocksPerWeek (window for a week)
        self.slashwindow = 100800                  # core.BlocksPerYear (window for a year)

        self.votethreshold = 0.500000000000000000  # 50%
        self.rewardband = 0.02                     # 2%
        
        self.whitelist = {"name": "ukrw", "tobin_tax": 0.002000000000000000}
        self.slashfraction = 0.001000000000000000                 # 0.01%
        self.minvalidperwindow = 0.050000000000000000             # 5%
        
        self.currentExchangeRates = {
            "uusd": 0,
            "usdr": 0,
            "ukrw": 0
        }
        self.toblinTax = {
            "uusd": 0.0035,
            "usdr": 0.0035,
            "ukrw": 0.0035,
            "umnt": 0.02,
            "ueur": 0.0035,
            "ucny": 0.0035,
            "ujpy": 0.0035,
            "ugbp": 0.0035,
            "uinr": 0.0035,
            "ucad": 0.0035,
            "uchf": 0.0035,
            "uhkd": 0.0035,
            "usgd": 0.0035,
            "uaud": 0.0035,
            "uthb": 0.0075,
            "usek": 0.0035,
            "udkk": 0.0035,
            "unok": 0.0035,
            "uidr": 0.0075,
            "uphp": 0.0075,
            "umyr": 0.0035,
            "utwd": 0.0035,
        }
    
        self.MicroLunaDenom = "uluna"
        self.MicroUSDDenom  = "uusd"
        self.MicroKRWDenom  = "ukrw"
        self.MicroSDRDenom  = "usdr"
        self.MicroCNYDenom  = "ucny"
        self.MicroJPYDenom  = "ujpy"
        self.MicroEURDenom  = "ueur"
        self.MicroGBPDenom  = "ugbp"
        self.MicroMNTDenom  = "umnt"    
    
    
    # EXTERNAL FUNCTION
    # get exchange rate
    def GetLunaExchangeRate(self,denom):
        if denom == self.MicroLunaDenom:
            return 1, None
        # retrieve exchange rate
        ex = self.currentExchangeRates[denom]
        if ex == None:
            return 0, "Unknown Denom"    
        return ex, None

    # INTERNAL FUNCTION
    # set exchange rate
    def SetLunaExchangeRate(self, denom, exchangeRate): 
        self.currentExchangeRates[denom] = exchangeRate
       
    
    # INTERNAL FUNCTION
    # delete exchange rate
    def _DeleteLunaExchangeRate(self, denom): 
        self.currentExchangeRates[denom] = 0
           
    # EXTERNAL FUNCTION -- Get Toblin Tax
    def GetTobinTax(self, denom):
        return self.toblinTax[denom]

    # EXTERNAL FUNCTION -- Set Toblin Tax
    def SetTobinTax(self, denom, tobinTax): 
        self.toblinTax[denom] = tobinTax
   

## Terra Classic : Fetch All Proposals 

In [95]:
# import json

In [96]:
# f = open("./simulated_datasets/final_proposals.json")
# proposals = json.load(f)
# # ind = 0


# id_ = 0

# type_ = ""
# description = ""
# title = ""
# recipient = ""
# changes = ""


# status = 0
# submit_time = ""
# deposit_end_time = ""
# voting_start_time = ""
# voting_end_time = ""
# total_deposit = 0

# voted_yes = 0
# voted_no = 0
# voted_no_with_veto = 0
# voted_abstain = 0


# description = ""
# description = ""



# # Proposals DF
# proposals_DF = pd.DataFrame(columns=["id","type","title","description","title","recipient","status","submit_time",\
#                                      "deposit_end_time","voting_start_time","voting_end_time","total_deposit","voted_yes",\
#                                     "voted_no","voted_no_with_veto","voted_abstain"])

# for proposal in proposals:

# #     ind = ind+1
# #     print(proposal)
#     content = proposal['content']
#     content_json = json.loads(content)
        
# #     keys = content_json.keys()
# #     for key in keys:
# #         if key not in ["description","title","recipient","changes","@type","amount"] :
# #             print(key)
                    
#     id_ = proposal["id"]
#     type_ = content_json["@type"]
#     description = content_json["description"]
#     title = content_json["title"]

#     if content_json.get("recipient"):
#         recipient = content_json["recipient"]
#     else:
#         recipient = 0

#     if content_json.get("changes"):
#         changes = content_json["changes"]
#     else:
#         changes = 0

#     status = proposal["status"]
#     submit_time = proposal["submit_time"]
#     deposit_end_time = proposal["deposit_end_time"]
#     voting_start_time = proposal["voting_start_time"]
#     voting_end_time = proposal["voting_end_time"]
#     total_deposit = proposal["total_deposit"]

#     voted_yes = proposal["final_tally_result"]["yes"]
#     voted_no = proposal["final_tally_result"]["no"]
#     voted_no_with_veto = proposal["final_tally_result"]["no_with_veto"]
#     voted_abstain = proposal["final_tally_result"]["abstain"]
    
#     proposals_DF.loc[len(proposals_DF.index) - 1] = [id_, type_,title, description,title,recipient,status,submit_time,\
#                                      deposit_end_time,voting_start_time,voting_end_time,total_deposit,voted_yes,\
#                                     voted_no,voted_no_with_veto,voted_abstain]

# #     if ind > 5:
# #         break
# # proposals = pd.read_json("./terra_classic_dataset/proposals.json")







In [97]:
# print(f"Total proposals = {len(proposals_DF.index)}")

In [98]:
# proposals_DF.columns

In [99]:
# proposals_DF.head(5)

In [100]:
proposals_DF = pd.read_csv("./simulated_datasets/terra_proposals.csv")

In [101]:
import datetime
import calendar


def status_mapping(x):
    if x == 0:
        return "UNSPECIFIED"
    if x == 1:
        return "DEPOSIT_PERIOD"
    if x == 2:
        return "VOTING_PERIOD"
    if x == 3:
        return "PASSED"
    if x == 4:
        return "REJECTED"
    if x == 5:
        return "FAILED"
    if x == -1:
        return "UNRECOGNIZED"

def covert_datetime(x):
    if x=="0001-01-01T00:00:00.000Z":
        return 0
    return pd.to_datetime(x)

proposals_DF['submit_time'] = proposals_DF.apply(lambda x: covert_datetime(x["submit_time"]), axis=1 )
proposals_DF['deposit_end_time'] = proposals_DF.apply(lambda x: covert_datetime(x["deposit_end_time"]), axis=1 )
proposals_DF['voting_start_time'] = proposals_DF.apply(lambda x: covert_datetime(x["voting_start_time"]), axis=1 )
proposals_DF['voting_end_time'] = proposals_DF.apply(lambda x: covert_datetime(x["voting_end_time"]), axis=1 )

proposals_DF['voted_yes'] = proposals_DF.apply(lambda x: int(x["voted_yes"])/10**6, axis=1 )
proposals_DF['voted_no'] = proposals_DF.apply(lambda  x: int(x["voted_no"])/10**6, axis=1 )
proposals_DF['voted_no_with_veto'] = proposals_DF.apply(lambda  x: int(x["voted_no_with_veto"])/10**6, axis=1 )
proposals_DF['voted_abstain'] = proposals_DF.apply(lambda  x: int(x["voted_abstain"])/10**6, axis=1 )

proposals_DF['status'] = proposals_DF.apply(lambda  x: status_mapping(x["status"]), axis=1 )

ParserError: day is out of range for month: 0

In [102]:
proposals_DF.head(5)

,Unnamed: 0,id,type,title,description,title.1,recipient,status,submit_time,deposit_end_time,voting_start_time,voting_end_time,total_deposit,voted_yes,voted_no,voted_no_with_veto,voted_abstain
0,-1,2,/cosmos.gov.v1beta1.TextProposal,"Unlocking the pre-seed, seed round's Luna Tokens at once.","There are countless questions about ""until when the great many of the pre-seed and seed round's Luna Tokens will be released?"" from small investors in korea and this becomes the FUD about price drop (caused by the dumping of the pre-seed and seed round investors, which will not happen) continues to spread over the time, causing existing cryptocurrency traders to continue to be reluctant to enter.\nIf all pre-seed and seed round's Luna tokens are released at once and show there is no drop in price and the staking of Luna Tokens by pre-seed, seed investors continues, as has been the case with the ATOM, small holders and small traders are expected to continue to believe in the Terra Project and continue to stake, hold their Luna Tokens and even want to buy more, which would lead proper value of Luna Tokens.\n\nOr if many of investors really just want to sell their tokens, not interested in what Terra has done, it would be better to allow them to sell early and let them go away from Terra Project, and give chances to small investors to buy Luna cheaper, not with concern about dumping but with a fresh start.","Unlocking the pre-seed, seed round's Luna Tokens at once.",0,REJECTED,2020-01-07 17:50:30.267000+00:00,2020-01-21 17:50:30.267000+00:00,2020-01-10 23:46:08.513000+00:00,2020-01-24 23:46:08.513000+00:00,"[{""amount"":""512000001"",""denom"":""uluna""}]","13,888,075.0840690","55,813,322.7610490",0.0000000,"7,280.4770510"
1,0,3,/cosmos.gov.v1beta1.TextProposal,Explicit disclosure of the circulating supply of LUNA,"The circulating supply of LUNA has always been a mystery in the Terra community. Coinmarketcap and Coingecko did their own calculation, but the figure differed from 189million to 287 million. And people from the community said the actual circulating supply was around 70million. \n An official report of the circulating supply can increase the transparency of information disclosure. In additon, the exaggerated market cap could be a major reason that prevented people from investing LUNA. If the actualy circulating supply is different from those released by coinmarketcap and coingecko, I hope the team can talk to these two platforms and modify this mistake.",Explicit disclosure of the circulating supply of LUNA,0,REJECTED,2020-01-11 05:08:03.810000+00:00,2020-01-25 05:08:03.810000+00:00,2020-01-11 05:08:30.272000+00:00,2020-01-25 05:08:30.272000+00:00,"[{""amount"":""513000000"",""denom"":""uluna""}]","6,366,306.9157040","44,395,896.6998940","5,117,483.3040850","14,759,543.8293390"
2,1,4,/cosmos.params.v1beta1.ParameterChangeProposal,Proposal to Implement a Temporary Tax Rate Freeze,Propose a temporary freeze in the tax rate until sufficient reward history has been established.\n\n-=-=-\n\nFull proposal: https://agora.terra.money/t/proposal-to-implement-a-temporary-tax-rate-freeze/172\n\n-=-=-\n\n,Proposal to Implement a Temporary Tax Rate Freeze,0,PASSED,2020-01-22 08:15:38.341000+00:00,2020-02-05 08:15:38.341000+00:00,2020-01-22 08:15:38.341000+00:00,2020-02-05 08:15:38.341000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","158,197,461.6313860",0.0000000,"441,787.7071060",0.0000000
3,2,5,/cosmos.gov.v1beta1.TextProposal,Sort validators by self-delegation (skin in the game),Option to sort validators by self-delegation percentage in the staking section of Terra Station.\n\nThe percentage of Luna that a validator is prepared to self-delegate shows how much she is prepared to risk on her own operation and is an important signal for delegators choosing validators.,Sort validators by self-delegation (skin in the game),0,REJECTED,2020-02-12 19:49:27.376000+00:00,2020-02-26 19:49:27.376000+00:00,2020-02-18 06:48:04.451000+00:00,2

In [ ]:
proposals_DF['type'].unique()

In [103]:
proposals_DF[proposals_DF["type"] == "/cosmos.params.v1beta1.ParameterChangeProposal" ]

,Unnamed: 0,id,type,title,description,title.1,recipient,status,submit_time,deposit_end_time,voting_start_time,voting_end_time,total_deposit,voted_yes,voted_no,voted_no_with_veto,voted_abstain
2,1,4,/cosmos.params.v1beta1.ParameterChangeProposal,Proposal to Implement a Temporary Tax Rate Freeze,Propose a temporary freeze in the tax rate until sufficient reward history has been established.\n\n-=-=-\n\nFull proposal: https://agora.terra.money/t/proposal-to-implement-a-temporary-tax-rate-freeze/172\n\n-=-=-\n\n,Proposal to Implement a Temporary Tax Rate Freeze,0,PASSED,2020-01-22 08:15:38.341000+00:00,2020-02-05 08:15:38.341000+00:00,2020-01-22 08:15:38.341000+00:00,2020-02-05 08:15:38.341000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","158,197,461.6313860",0.0000000,"441,787.7071060",0.0000000
5,4,7,/cosmos.params.v1beta1.ParameterChangeProposal,Increase Minimum Deposit for Goverance Proposal,Proposal to increase minimum deposit required for goverance proposal submit.\n\n-=-=-\n\nFull proposal: https://agora.terra.money/t/proposal-to-increase-minimum-deposit-for-governance-proposals/214\n\n-=-=-\n\n,Increase Minimum Deposit for Goverance Proposal,0,REJECTED,2020-03-31 09:30:16.580000+00:00,2020-04-14 09:30:16.580000+00:00,2020-03-31 09:30:16.580000+00:00,2020-04-14 09:30:16.580000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","91,977,336.3185880","7,475.7388930",0.0000000,"3,683,531.0400010"
6,5,8,/cosmos.params.v1beta1.ParameterChangeProposal,Decrease Goverance Minimum Quorum,Proposal to decrease minimum quorum size required for goverance proposal passed.\n\n-=-=-\n\nFull proposal: https://agora.terra.money/t/proposal-to-decrease-governance-quorum-to-20/215\n\n-=-=-\n\n,Decrease Goverance Minimum Quorum,0,REJECTED,2020-03-31 09:34:37.232000+00:00,2020-04-14 09:34:37.232000+00:00,2020-03-31 09:34:37.232000+00:00,2020-04-14 09:34:37.232000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","74,513,274.6745650","8,878,478.0197940","1,579,063.8493360",0.0000000
8,7,10,/cosmos.params.v1beta1.ParameterChangeProposal,Temporary Increasing Tobin Tax From 0.25% to 0.35%,"Temporary increasing Tobin tax from 0.25% to 0.35%, to protect the network from arbitrage attacks which are allowed by vulnerable terra-vs-terra token cross-rate computation.\nIPFS Link : https://ipfs.io/ipfs/Qme2UmaSaEFc5jx8fk1my8TFrq73Cv9tZpisAPwWbLzTGQ?filename=Terra(columbus-3)_gov_proposal_%2310.pdf\nForum Link : https://agora.terra.money/t/governance-temporarily-increase-tobin-tax-from-0-25-to-0-35/233",Temporary Increasing Tobin Tax From 0.25% to 0.35%,0,PASSED,2020-06-07 16:35:38.324000+00:00,2020-06-21 16:35:38.324000+00:00,2020-06-07 16:35:38.324000+00:00,2020-06-21 16:35:38.324000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","214,477,373.1695820","158,795.5404840",0.0000000,0.0000000
9,8,11,/cosmos.params.v1beta1.ParameterChangeProposal,Proposal to decrease minspread of Terra to Luna swaps,We propose to change minspread value of Terra to Luna swaps from 2% to 0.5%.\n\nForum Link: https://agora.terra.money/t/proposal-to-decrease-minspread-of-terra-luna-swaps/244,Proposal to decrease minspread of Terra to Luna swaps,0,PASSED,2020-09-02 08:50:55.038000+00:00,2020-09-16 08:50:55.038000+00:00,2020-09-02 08:50:55.038000+00:00,2020-09-16 08:50:55.038000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","252,895,555.0224390",0.0000000,0.0000000,0.0000000
10,9,12,/cosmos.params.v1beta1.ParameterChangeProposal,Proposal to increase basepool value,"We propose to change the basepool value of Terra to Luna swaps from 250,000 SDR to 625,000 SDR.\n\nForum Link: https://agora.terra.money/t/proposal-to-increase-base-pool-value/245",Proposal to increase basepool value,0,PASSED,2020-09-02 09:00:23.173000+00:00,2020-09-16 09:00:23.173000+00:00,2020-09-02 09:00:23.173000+00:00,2020-09-16 09:00:23.173000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","233,700,193.9831720",0.0000000,0.0000000,0.0000000
13,12,15,/cosmos.params

In [105]:
proposals_DF["type"].unique()

array(['/cosmos.gov.v1beta1.TextProposal',
       '/cosmos.params.v1beta1.ParameterChangeProposal',
       '/cosmos.distribution.v1beta1.CommunityPoolSpendProposal',
       '/ibc.core.client.v1.ClientUpdateProposal'], dtype=object)

In [106]:
proposals_DF[proposals_DF["type"] == "/cosmos.distribution.v1beta1.CommunityPoolSpendProposal" ]

,Unnamed: 0,id,type,title,description,title.1,recipient,status,submit_time,deposit_end_time,voting_start_time,voting_end_time,total_deposit,voted_yes,voted_no,voted_no_with_veto,voted_abstain
4,3,6,/cosmos.distribution.v1beta1.CommunityPoolSpendProposal,Seigniorage Allocation to Chai,"Chai is a payments application that works in close proximity with Terra. After its launch in June of 2019, it has seen tremendous growth by partnering with major commerce players in Korea such as TMON (general e-Commerce), Yanolja (hospitality and leisure), and CU (#1 convenience store with over 14000 stores across Korea). Some key metrics for the project are as follows: 1M+ users accumulated in less than a year, $400M in expected annual transactions (run-rate), integrations with 12+ major commerce players in Korea with many more to come. Chai's transaction volumes are reflected on the Terra blockchain, meaning that The transaction volumes of the Chai payment application are correlated with Terra transaction volumes.\nChai utilizes seigniorage-funded discounts to offer deals for customers who choose Chai at merchant checkout. This creates a virtuous cycle where (1) users select Chai at a merchant's checkout page (2) Chai/Terra tx volume increases which in turn increases Terra money supply (seigniorage creation) (3) seigniorage is allocated to Chai which then uses it as a budget for providing promotion discounts (4) Users are incentivized to use Chai at checkout due to seigniorage-driven promotional schemes.\nGiven the scale of current Chai transaction volumes, its potential to capture even more volume through continued business growth, and its clear value proposition for both the Terra ecosystem and users shoppping for everyday goods and services, it is our firm belief that Chai provides the highest ROI for seigniorage spent at this juncture.",Seigniorage Allocation to Chai,terra1cjsf3lf8ryyj7jrraythrjkzy6y27graqvw8x9,PASSED,2020-03-18 07:09:58.463000+00:00,2020-04-01 07:09:58.463000+00:00,2020-03-18 07:09:58.463000+00:00,2020-04-01 07:09:58.463000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","99,368,236.5606220","8,411,516.4156370",0.0000000,"5,221,812.9265290"
21,20,35,/cosmos.distribution.v1beta1.CommunityPoolSpendProposal,RPC Endpoint and API Infrastructure Proposal,Proposer: Blockdaemon\nhttps://agora.terra.money/t/infrastructure-proposal-for-community-pool-funding/320,RPC Endpoint and API Infrastructure Proposal,terra1nzk3kzf0at2lfzrc6ud8a7d340gd9pkrdkz8f9,PASSED,2021-02-09 15:27:49.740000+00:00,2021-02-23 15:27:49.740000+00:00,2021-02-09 15:27:49.740000+00:00,2021-02-23 15:27:49.740000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","166,726,046.8063450",0.0000000,0.0000000,"36,166.9160000"
27,26,44,/cosmos.distribution.v1beta1.CommunityPoolSpendProposal,TIP 44: Burn the community pool,https://agora.terra.money/t/proposal-to-burn-all-seigniorage/438,TIP 44: Burn the community pool,terra1luk43x0g9vva7ws7ju9cl7g206wmeafzxl7vpz,PASSED,2021-03-03 10:12:40.960000+00:00,2021-03-17 10:12:40.960000+00:00,2021-03-03 10:12:40.960000+00:00,2021-03-17 10:12:40.960000+00:00,"[{""amount"":""512000000"",""denom"":""uluna""}]","149,594,235.8403360","61,278.9469640",0.0000000,"34,591.8079220"
29,28,51,/cosmos.distribution.v1beta1.CommunityPoolSpendProposal,Proposal: PayWithTerra Community Pool Funding,"We need some funds to revision of the system and plugins (coding), design and illustrations, legal. Detailed description on topic:\nhttps://agora.terra.money/t/proposal-paywithterra-community-pool-funding/492",Proposal: PayWithTerra Community Pool Funding,terra106h043ewfd0xrnwuazkf05q72cfm7kwj784la2,PASSED,2021-03-17 18:08:22.342000+00:00,2021-03-31 18:08:22.342000+00:00,2021-03-22 03:21:34.714000+00:00,2021-04-05 03:21:34.714000+00:00,"[{""amount"":""523017149"",""denom"":""uluna""}]","159,771,854.9538170",0.0000000,0.0000000,"3,000.0000000"
34,33,67,/cosmos.distribution.v1beta1.CommunityPoolSpendProposal,LocalTerra - Development - Comu

### Market module parameter update Proposals Summary - 

**Proposal #10 :: Temporary Increasing Tobin Tax From 0.25% to 0.35** PASSED (2020-06-07)
Forum - https://classic-agora.terra.money/t/governance-temporarily-increase-tobin-tax-from-0-25-to-0-35/233
Summary of the issue

- Due to an algorithmic flaw in computation of terra-vs-terra swap rate, there exists very frequent short-term swap rate volatility
- The short term swap rate volatility allows arbitrage trader to create significant profit in short term
related blog post : (Terra SDR Arbitrage and Cross-rate Calculation | by B-Harvest)[https://bharvest.medium.com/terra-sdr-arbitrage-and-cross-rate-calculation-a77bfee70ca0]
- A trader executing the strategy : (Terra (LUNA) Blockchain Explorer by Staking Fund 25)[]


**Proposal #11 :: Proposal to decrease minspread of Terra to Luna swaps from 2% to 0.5%** PASSED (2020-09-02)
Forum - https://classic-agora.terra.money/t/proposal-to-decrease-minspread-of-terra-luna-swaps/244
Summary
- Propose to change minspread value of Terra <> Luna swaps from 2% to 0.5%
- By decreasing minspread value, we expect there will be more Luna to be burned after Col-4 update

**Proposal #12 :: 	Proposal to increase basepool value from 250,000 SDR to 625,000 SDR** PASSED (2020-09-02)
Forum - https://classic-agora.terra.money/t/proposal-to-increase-base-pool-value/245
Summary
- Propose to change base_pool value of Terra <> Luna swaps from 250,000 SDR to 625,000 SDR

 
**Proposal #18 :: 	Proposal to decrease PoolRecoveryPeriod**  PASSED (2020-11-20)
Forum - https://classic-agora.terra.money/t/proposal-to-decrease-poolrecoveryperiod-to-8hour-4800-block/261
Summary
- Propose to change PoolRecoveryPeriod from 24hr(14400) to 8hr(4800)
- By decreasing PoolRecoveryPeriod, native swap liquidity increases while defending front-running attacks at the same level



**Proposal #27 :: 	Making Terra swaps more capital efficient**  PASSED (2021-01-25)
Forum - https://classic-agora.terra.money/t/terra-on-chain-liquidity-parameters/305



**Proposal #36 :: Further improvements to minting parameters** PASSED (2021-02-10)
Forum - https://classic-agora.terra.money/t/tip-36-further-improvements-to-liquidity-parameters/372
Summary
- We propose further increasing the minting parameters to:
    - Basepool 7M → 13M SDT
    - PoolRecoveryPeriod 200 → 130 blocks





**Proposal #90 :: Further improvements to minting parameters** PASSED (2021-05-23)
Forum - https://classic-agora.terra.money/t/liquidity-parameters-2/1175
Summary
- Increase BasePool size to 32,500,000 SDR
- Reduce PoolRecoveryPeriod to 49 blocks
- Split out TerraPoolDelta into TerraPoolDeltaBid and TerraPoolDeltaAsk.


**Proposal #185	:: Improvements to Liquidity/Minting Parameters** PASSED (2022-02-02)
Forum - https://classic-agora.terra.money/t/liquidity-parameters-3/3895
- Given the rapid changes in the UST ecosystem, the on-chain liquidity parameters should be updated to preserve stability, in the following ways:

    - Increase BasePool size to 50,000,000 SDR
    - Reduce PoolRecoveryPeriod to 36 blocks


**Proposal #1164 :: Help UST Pegging - Adjustments to Mint/Burn Parameters** PASSED (2022-05-11)
Forum - https://classic-agora.terra.money/t/proposal-help-ust-pegging-increase-estimated-minting-capacity-to-1200m/6287

Summary
- Increase BasePool from 50M to 100M SDR
- Decrease PoolRecoveryBlock from 36 to 18 Blocks
Link - https://jp12.medium.com/understanding-terras-market-module-and-redemption-capacity-5bd62e2d7879


**Proposal #1169 ::	Emergency measures for restoring Terra peg** REJECTED (2022-05-1)
Forum - https://classic-agora.terra.money/t/proposal-help-ust-pegging-increase-estimated-minting-capacity-to-1200m/6287

Proposal is to:
- Increase BasePool from 50M to 100M SDR
- Decrease PoolRecoveryBlock from 36 to 18 Blocks
Link - https://jp12.medium.com/understanding-terras-market-module-and-redemption-capacity-5bd62e2d7879







In [ ]:
proposals_DF[proposals_DF["id"] == 27]

In [ ]:
proposals_DF.to_csv("terra_proposals.csv")


### Market Module base_pool parameter history
    -  2020-09-02 : 250,000 SDR to 625,000 SDR
    - 2021-02-10 : Basepool 7M → 13M SDT
    - 2022-05-23 : Increase BasePool size to 32,500,000 SDR
    - 2022-02-02 : Increase BasePool size to 50,000,000 SDR
    - 2022-05-11 : Increase BasePool  from 50M to 100M SDR [After crash]
    
- Basepool 7M → 13M SDT
PoolRecoveryPeriod 200 → 130 blocks

### Market Module min_spread parameter history
    - 2020/09/02 : changed from 2% to 0.5%


### Market Module pool_recovery_period parameter history
    - 2020-11-20 : Change PoolRecoveryPeriod from 24hr(14400) to 8hr(4800)
    - 2021-02-10 : PoolRecoveryPeriod 200 → 130 blocks
    - 2021-05-23 : Reduce PoolRecoveryPeriod to 49 blocks
    - 2022-02-02 : Reduce PoolRecoveryPeriod to 36 blocks
    - 2022-05-11 : Reduce PoolRecoveryPeriod to 18 blocks [After crash]




Mentioned by Jump - Split out TerraPoolDelta into TerraPoolDeltaBid and TerraPoolDeltaAsk.


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import csv
# import math

# # https://classic-agora.terra.money/t/seigniorage-distribution-framework/212

# """
# Seigniorage allocation simulation in a 2 firm economy(dappA, dappB)
# Inputs
# Tax Spent(include swap fee), Total Value Locked timeseries data for each firm
# Parameters
# lambda
# alpha
# Outputs
# Funding Weight timeseies for each firm
# """

# class SeigniorageState:
#     def __init__(self, dappA, dappB, λ, α):
#         self.dappA = dappA
#         self.dappB = dappB
#         self.λ = λ
#         self.α = α

#     # tax Spent, total value locked
#     # 1 period = 14days
#     def Wil(self):
#         wil = []
#         j = 0
#         for i in range(0, len(self.dappA), 14):
#             periodic_dappA_fee = 0
#             periodic_dappA_tvl = 0
#             periodic_dappB_fee = 0
#             periodic_dappB_tvl = 0
#             temp_dappA = self.dappA[i : i + 14]
#             temp_dappB = self.dappB[i : i + 14]

#             dappA_wil = 0
#             dappB_wil = 0

#             if len(self.dappA[i : i + 14]) != 14:
#                 break

#             # calculate periodic value
#             for i in range(len(temp_dappA)):
#                 periodic_dappA_fee = periodic_dappA_fee + float(temp_dappA[i][1])
#                 periodic_dappA_tvl = periodic_dappA_tvl + math.sqrt(
#                     float(temp_dappA[i][2])
#                 )
#                 periodic_dappB_fee = periodic_dappB_fee + float(temp_dappB[i][1])
#                 periodic_dappB_tvl = periodic_dappB_tvl + math.sqrt(
#                     float(temp_dappB[i][2])
#                 )

#             # calculate wil value using periodic value
#             dappA_wil = self.λ * (
#                 periodic_dappA_fee / (periodic_dappA_fee + periodic_dappB_fee)
#             ) + (1 - self.λ) * (
#                 periodic_dappA_tvl / (periodic_dappA_tvl + periodic_dappB_tvl)
#             )
#             dappB_wil = self.λ * (
#                 periodic_dappB_fee / (periodic_dappA_fee + periodic_dappB_fee)
#             ) + (1 - self.λ) * (
#                 periodic_dappB_tvl / (periodic_dappA_tvl + periodic_dappB_tvl)
#             )

#             wil.append([j, dappA_wil, dappB_wil])
#             j = j + 1

#         return wil

#     # growth
#     def Wig(self):
#         wig = []
#         j = 0

#         for i in range(0, len(self.dappA), 14):
#             temp_dappA = self.dappA[i : i + 14]
#             temp_dappB = dappB[i : i + 14]
#             periodic_dappA_fee_growth = 0
#             periodic_dappB_fee_growth = 0
#             periodic_dappA_tvl_growth = 0
#             periodic_dappB_tvl_growth = 0

#             if len(self.dappA[i : i + 14]) != 14:
#                 break

#             # calculate the sum of TVL growth rate
#             for i in range(len(temp_dappA)):
#                 try:
#                     periodic_dappA_fee_growth = periodic_dappA_fee_growth + (
#                         (float(temp_dappA[i + 1][1]) - float(temp_dappA[i][1]))
#                         / float(temp_dappA[i][1])
#                     )
#                     periodic_dappA_tvl_growth = periodic_dappA_tvl_growth + (
#                         (
#                             math.sqrt(float(temp_dappA[i + 1][2]))
#                             - math.sqrt(float(temp_dappA[i][2]))
#                         )
#                         / math.sqrt((float(temp_dappA[i][2])))
#                     )
#                     periodic_dappB_fee_growth = periodic_dappB_fee_growth + (
#                         (float(temp_dappB[i + 1][1]) - float(temp_dappB[i][1]))
#                         / float(temp_dappB[i][1])
#                     )
#                     periodic_dappB_tvl_growth = periodic_dappB_tvl_growth + (
#                         (
#                             math.sqrt(float(temp_dappB[i + 1][2]))
#                             - math.sqrt(float(temp_dappB[i][2]))
#                         )
#                         / math.sqrt(float(temp_dappB[i][2]))
#                     )
#                 except:
#                     pass

#             # divide TVL growth rate sum by 13, and take max function
#             periodic_dappA_fee_growth = max(0, periodic_dappA_fee_growth / 13)
#             periodic_dappA_tvl_growth = max(0, periodic_dappA_tvl_growth / 13)
#             periodic_dappB_fee_growth = max(0, periodic_dappB_fee_growth / 13)
#             periodic_dappB_tvl_growth = max(0, periodic_dappB_tvl_growth / 13)

#             # the denominator can't be 0
#             if (
#                 periodic_dappA_fee_growth + periodic_dappB_fee_growth == 0
#                 or periodic_dappA_tvl_growth + periodic_dappB_tvl_growth == 0
#             ):
#                 dappA_wig = 0
#                 dappB_wig = 0

#             # calculate wig using periodic value
#             else:
#                 dappA_wig = self.λ * (
#                     periodic_dappA_fee_growth
#                     / (periodic_dappA_fee_growth + periodic_dappB_fee_growth)
#                 ) + (1 - self.λ) * (
#                     periodic_dappA_tvl_growth
#                     / (periodic_dappA_tvl_growth + periodic_dappB_tvl_growth)
#                 )
#                 dappB_wig = self.λ * (
#                     periodic_dappB_fee_growth
#                     / (periodic_dappA_fee_growth + periodic_dappB_fee_growth)
#                 ) + (1 - self.λ) * (
#                     periodic_dappB_tvl_growth
#                     / (periodic_dappA_tvl_growth + periodic_dappB_tvl_growth)
#                 )

#             wig.append([j, dappA_wig, dappB_wig])

#             j = j + 1

#         return wig

#     def funding_weight(self, wil, wig):
#         w = []

#         for i in range(len(wil)):
#             dappA_w = 0
#             dappB_w = 0
#             dappA_w = self.α * wil[i][1] + (1 - self.α) * wig[i][1]
#             dappB_w = self.α * wil[i][2] + (1 - self.α) * wig[i][2]
#             w.append([dappA_w, dappB_w])

#         w = pd.DataFrame(w)
#         w = w.rename(columns={0: "dappA", 1: "dappB"})

#         return w
    
#     def create_graph(self, w):
#         df = w.divide(w.sum(axis=1), axis=0)
#         ax = df.plot(
#             kind="area",
#             color=["lightcoral", "skyblue"],
#             stacked=True,
#             title="λ={}  α={}".format(λ, α)
#         )

#         ax.set_xlabel("Period (2 weeks)")
#         ax.set_ylabel("Funding Weight")
#         ax.set_xlim(0, 25)

#         return plt.show()
        


# if __name__ == "__main__":
#     with open("dappA.csv", newline="") as f:
#         reader = csv.reader(f)
#         dappA = list(reader)
#         dappA = dappA[1:]

#     with open("dappB.csv", newline="") as g:
#         reader = csv.reader(g)
#         dappB = list(reader)
#         dappB = dappB[1:]

#     # read λ and α from the command line
#     λ = float(input("funding weight parameter lambda :"))
#     α = float(input("funding weight parameter alpha :"))

#     if α < 0 or α > 1 or λ < 0 or λ > 1:
#         raise ValueError("lambda and alpha must be between 0 and 1 inclusive")

#     seign = SeigniorageState(dappA=dappA, dappB=dappB, λ=λ, α=α) 
#     w = seign.funding_weight(seign.Wil(), seign.Wig())
    
#     seign.create_graph(w)